In [1]:
#importation des librairies
import pandas as pd #matrice
import sklearn as sk #Machine Learning
import numpy as np

from datetime import datetime
import time

from math import *
import statistics

from joblib import dump, load

#from blog saladtomatonion.com
class Timer(object):
    def __enter__(self):
        self.start()
        return self
    
    def __exit__(self, *args, **kwargs):
        self.stop()
        
    def start(self):
        if hasattr(self, 'interval'):
            del self.interval
        self.start_time = time.time()
    
    def stop(self):
        if hasattr(self, 'start_time'):
            self.interval = time.time() - self.start_time
            del self.start_time
            
def formatDFsansOthers(df):
    #mise en minuscule
    df["Entité"] = df["Entité"].str.lower()
    df["Compte"] = df["Compte"].str.lower()
    df["Fournisseur identification"] = df["Fournisseur identification"].str.lower()
    df["Cd_Scat"] = df["Cd_Scat"].str.lower()
    df["Fournisseur"] = df["Fournisseur"].str.lower()
    df["Family Stream"] = df["Family Stream"].str.lower()
    df["Category Stream"] = df["Category Stream"].str.lower()
    df["Sub Category Stream"] = df["Sub Category Stream"].str.lower()
    
    #suppression des lignes non voulues pour l'apprentissage :
    #les others qui ne sont pas des "transactions fees"
    #EN COMMENTAIRE POUR L'APPLICATION AU RUN et POUR POUVOIR APPRENDRE LES OTHERS EGALEMENT
    indexNames = df[(df['Family Stream'] == 'others') & (df['Category Stream'] != 'transaction fees')].index
    df.drop(indexNames , inplace=True)

    #remplis les vide avec un tiret "-"
    df.fillna("-", inplace=True)

    #aggrega
    df["AcléMin"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]
    df["AcléMinE"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]

    df["AcléEt"]=df["Entité"]+" ; "+df["Compte"]+" ; "+df["Fournisseur identification"]+" ; "+df["Cd_Scat"]+" ; "+df["Fournisseur"]
    df["AcléEtSansE"]=df["Compte"]+" ; "+df["Fournisseur identification"]+" ; "+df["Cd_Scat"]+" ; "+df["Fournisseur"]
    #df["AcléTout"]=df["Entité"]+';'+df["CodePays"]+';'+df["Compte"]+';'+df["Sous_catégorie_de_dépenses"]+';'+df["Catégorie_de_dépenses"]+';'+df["Famille_de_dépenses"]+';'+df["Fournisseur"]+';'+df["Fournisseur identification"]

    df["Acible"]=df["Family Stream"]+" ; "+df["Category Stream"]+" ; "+df["Sub Category Stream"]
    
    return df

In [149]:
#fichier d'apprentissage/Learn
#utilise un fichier avec un seul onglet pour simplifier
dfL = pd.read_excel('./Cube_Global_View_2020_FY_v0.94 - pour ML.xlsx')

#sinon utiliser qq chose comme suit qui permet de spécifier l'onglet
#dfL = pd.read_excel('./Cube_Global_View_2020_FY_v0.94 - pour ML.xlsx', "Enriched Data", index_col=None, na_values=["NA"])
#ou en 2 temps
#xlsx = pd.ExcelFile('./Cube_Global_View_2020_FY_v0.94 - pour ML.xlsx')
#dfL = pd.read_excel(xlsx, "Enriched Data")

#suppression des colonnes non nécessaires,
#ou pouvant induire des résultats faux
#et pour travailler sur une copie
df2020 = dfL.copy() #.drop(columns=["MntKDevise", "SommeDekEUR", "Annee", "Période", "Devise", "Country", "Region"])

#fichier d'application/Restitution
dfR = pd.read_excel('./Cube 2021 Q1 v0.92.xlsx')
#pour travailler sur une copie
df2021Q1 = dfR.copy()
df2021Q1=df2021Q1.rename(columns={'kEUR': 'SommeDekEUR'})

dfGT = pd.read_excel('./Cube 2018 (V0.9 du 25.01.21).xlsx')
#pour travailler sur une copie
df2018 = dfGT.copy()
df2018=df2018.rename(columns={'kEUR': 'SommeDekEUR'})

#append des 2 dataframe (2020Q4 et 2021Q1)
df1921 = df2020.append(df2021Q1, ignore_index=True)
#append des 3 dataframe (2020Q4 et 2021Q1 et 2018Q4 (via 2019 Q4))
df1821 = df1921.append(df2018, ignore_index=True)

df1821 = formatDFsansOthers(df1821)
df1921 = formatDFsansOthers(df1921)

dump(df1821, 'df1821.dump')

In [2]:
df1821 = load('df1821.dump')

In [3]:
df1821

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Cd_Cat,Catégorie_de_dépenses,Cd_Fam,...,Fournisseur identification,SommeDekEUR,Family Stream,Category Stream,Sub Category Stream,AcléMin,AcléMinE,AcléEt,AcléEtSansE,Acible
0,PL45551,bnpp bank polska,PL,ROD.954M30,organizacja eventów i spotka?,rp,"Evénementiel, RP et services marketing",CSLTG,Prestations de conseil en communication,C&MK,...,""",,na brzegu"""" s.c. gastronomia i noc""",0.14400,technology,hardware,-,bnpp bank polska ; organizacja eventów i spotk...,bnpp bank polska ; organizacja eventów i spotk...,bnpp bank polska ; organizacja eventów i spotk...,"organizacja eventów i spotka? ; "",,na brzegu""""...",technology ; hardware ; -
1,ES018,uci espana,ES,UCI.62110,alquiler locales,loyer,Loyer,LOYER,Coûts d'occupation,IMMO,...,"""3491 ma, s.l""",82.32000,corporate services,corporate real estate,costs of occupancy,"uci espana ; alquiler locales ; ""3491 ma, s.l""","uci espana ; alquiler locales ; ""3491 ma, s.l""","uci espana ; alquiler locales ; ""3491 ma, s.l""...","alquiler locales ; ""3491 ma, s.l"" ; loyer ; ""3...",corporate services ; corporate real estate ; c...
2,FR084,bnpp personal finance,FR,OCAPP.634622,presta informatiques refacturees ips specifiqu...,amoe,Assistance à la maîtrise d'oeuvre,CONSU,"Conseil, assistance technique",SPRO,...,bnp paribas group,131501.51836,bnp paribas group,-,-,bnpp personal finance ; presta informatiques r...,bnpp personal finance ; presta informatiques r...,bnpp personal finance ; presta informatiques r...,presta informatiques refacturees ips specifiqu...,bnp paribas group ; - ; -
3,FR060,bnpp imex,FR,SAP_IMEX.I621100RE0,loyers locaux d'exp,loyer,Loyer,LOYER,Coûts d'occupation,IMMO,...,bnp paribas group,114859.49123,bnp paribas group,-,-,bnpp imex ; loyers locaux d'exp ; bnp paribas ...,bnpp imex ; loyers locaux d'exp ; bnp paribas ...,bnpp imex ; loyers locaux d'exp ; bnp paribas ...,loyers locaux d'exp ; bnp paribas group ; loye...,bnp paribas group ; - ; -
4,FR084,bnpp personal finance,FR,OCAPP.634622,presta informatiques refacturees ips specifiqu...,amoe,Assistance à la maîtrise d'oeuvre,CONSU,"Conseil, assistance technique",SPRO,...,bnp paribas group,108750.00256,bnp paribas group,-,-,bnpp personal finance ; presta informatiques r...,bnpp personal finance ; presta informatiques r...,bnpp personal finance ; presta informatiques r...,presta informatiques refacturees ips specifiqu...,bnp paribas group ; - ; -
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469482,IN43974,arval india private ltd,-,-,staff - company celebration,asoc0,z-Avantages sociaux - autres,A-SOC,Avantages sociaux,SPRO,...,-,0.16687,corporate insurance,-,-,arval india private ltd ; staff - company cele...,arval india private ltd ; staff - company cele...,arval india private ltd ; staff - company cele...,staff - company celebration ; - ; asoc0 ; -,corporate insurance ; - ; -
1469483,IN43974,arval india private ltd,-,-,prof. fees - legal fees,juri0,z-Honoraires juridiques - autres,JURID,Honoraires juridiques,SPRO,...,-,0.61958,corporate insurance,-,-,arval india private ltd ; prof. fees - legal f...,arval india private ltd ; prof. fees - legal f...,arval india private ltd ; prof. fees - legal f...,prof. fees - legal fees ; - ; juri0 ; -,corporate insurance ; - ; -
1469513,RU43677,cardif insurance company llc,-,-,opex marketing consulting and channels testing,c&mk0,z-Communication et Marketing - autres,C&MK0,Z-Communication et Marketing - autres,C&MK,...,??? ???? ?????? ? ????????????????,31.91862,professional services,non it consultancy (excl. emc),-,cardif insurance company llc ; opex marketing ...,cardif insurance company llc ; opex marketing ...,cardif insurance company llc ; opex marketing ...,opex marketing consulting and channels testing...,professional services ; non it consultancy (ex...
1469514,RU43677,cardif insurance company llc,-,-,opex other consulting,cons0,"z-Conseil, assistance te

In [9]:
df1821['sommeDeValAbsEUR'] = [sqrt(a*a) for a in df1821['SommeDekEUR']]
df1821['1erChiffreSignificatif'] = [str(a)[0] for a in df1821['sommeDeValAbsEUR']]

[[0.144, 0.144],
 [82.32, 82.32],
 [131501.51836, 131501.51836],
 [114859.49123, 114859.49123],
 [108750.00256, 108750.00256],
 [6.58, 6.58],
 [96757.33743, 96757.33743],
 [0.12, 0.12],
 [95417.84219, 95417.84219],
 [93541.70964, 93541.70964],
 [91577.77985, 91577.77985],
 [80938.84069, 80938.84069],
 [21.04296, 21.04296],
 [2.33161, 2.33161],
 [68048.8691, 68048.8691],
 [65597.63364, 65597.63364],
 [64344.72875, 64344.72875],
 [63182.16198, 63182.16198],
 [1.67155, 1.67155],
 [0.12, 0.12],
 [55956.39116, 55956.39116],
 [0.768, 0.768],
 [51816.4644, 51816.4644],
 [51625.89327, 51625.89327],
 [47034.74182, 47034.74182],
 [45090.45716, 45090.45716],
 [44746.67556, 44746.67556],
 [43862.69015, 43862.69015],
 [43696.13952, 43696.13952],
 [43291.90927, 43291.90927],
 [42632.23163, 42632.23163],
 [42032.4, 42032.4],
 [40962.225, 40962.225],
 [40601.40161, 40601.40161],
 [40567.82954, 40567.82954],
 [40074.2237, 40074.2237],
 [38644.0819, 38644.0819],
 [38028.62755, 38028.62755],
 [1.62728, 1

In [42]:
df1821['sommeDeValAbsEUR'] = 1000000000 * df1821['sommeDeValAbsEUR']

In [43]:
df1821['1erChiffreSignificatif'] = [str(a)[0] if str(a)[0] != '0' else str(a)[2] for a in df1821['sommeDeValAbsEUR']]

In [47]:
df1CS = df1821[df1821['sommeDeValAbsEUR'] != 0]

In [52]:
print(100 * df1CS.groupby('1erChiffreSignificatif')['CdEnt'].count() / sum(df1CS.groupby('1erChiffreSignificatif')['CdEnt'].count()))

1erChiffreSignificatif
1    30.150743
2    17.463658
3    12.370289
4     9.743715
5     8.032285
6     6.721570
7     5.894033
8     5.152720
9     4.470988
Name: CdEnt, dtype: float64


# Machine Learning

In [3]:
#début ML
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.impute import KNNImputer as KNNI
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import neighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neural_network import MLPClassifier

from nltk.corpus import stopwords

### préparation

In [4]:
class chaine():
    def __init__(self, val, score, temps, model):
        self.valeur = val
        self.score = score
        self.temps = temps
        self.suivant = None
        self.precedent = None
        self.model = model
    
    def setSuivant(self, chaineSuivante):
        self.suivant = chaineSuivante
        chaineSuivante.setPrecedent(self)
        
    def setPrecedent(self, chainePrecedente):
        self.precedent = chainePrecedente
    
    def placeAvant(self, nouvelleChainePrecedente):
        if self.precedent != None:
            self.precedent.setSuivant(nouvelleChainePrecedente)
        else:
            nouvelleChainePrecedente.precedent = None
            
        nouvelleChainePrecedente.setSuivant(self)
    
    def placeApres(self, nouvelleChaineSuivante):
        if self.suivant != None:
            nouvelleChaineSuivante.setSuivant(self.suivant)            
        else:
            nouvelleChaineSuivante.suivant = None
            
        self.setSuivant(nouvelleChaineSuivante)
    
def getMaillonScoreMax(maillon):
    maxS = -1
    placeC = 1
    while True:
        if maillon.score > maxS :
            maxS = maillon.score
            maxM = maillon
            place = placeC
        
        if maillon.suivant != None:
            maillon=maillon.suivant
            placeC = placeC + 1
        else:
            return maxM, place
            break

def newValAutourduMax(maillon):
    valC = maillon.valeur
    
    if maillon.precedent != None:
        valPrec = maillon.precedent.valeur
    else:
        valPrec = valC/10

    if maillon.suivant != None:
        valSuiv = maillon.suivant.valeur
    else:
        valSuiv = valC*10
    
    newVP = valC/sqrt(valC/valPrec)
    newVS = valC*sqrt(valSuiv/valC)
    #pour accélerer la recherche autour du point de départ
    #newVP = valC/sqrt(sqrt(valC/valPrec))
    #newVS = valC*sqrt(sqrt(valSuiv/valC))
    
    return (newVP, newVS)
    
def apprendData(valeurVarie, criterion, n_estimators, tol, solver, loss, class_weight, typeML, labelData, X_train_vectorized, X_test_vectorized, y_train, y_test):
    label = str(valeurVarie)
    
    if typeML=='LogisticRegression':
        model = LogisticRegression(solver=solver, class_weight=class_weight, tol=tol, C=valeurVarie, max_iter=1200)#, l1_ratio=0.1)
    elif typeML=='PassiveAggressiveClassifier':
        model = PassiveAggressiveClassifier(class_weight=class_weight, tol=tol, C=valeurVarie, max_iter=800)
    elif typeML=='SGDClassifier':
        model = SGDClassifier(class_weight=class_weight, loss=loss, tol=tol, alpha=valeurVarie, max_iter=400)#, l1_ratio=0.1)
    elif typeML=='LinearSVC':
        model = LinearSVC(class_weight=class_weight, loss=loss, tol=tol, C=valeurVarie, max_iter = 1200)#(max_iter = 1000, C=5.5)
    elif typeML=='RandomForest':
        model = RandomForestClassifier(class_weight=class_weight, n_estimators=n_estimators, n_jobs = 2, criterion=criterion, max_depth=valeurVarie)
    
    with Timer() as timer:
        modelC = model.fit(X_train_vectorized, y_train)
        #dicModel[typeML][labelData][label] = model.fit(X_train_vectorized, y_train)
    print(str(datetime.now())+" - valeur : "+str(valeurVarie)+" - fit en : "+str(timer.interval))

    #print('************ :'+str(X_test_vectorized.shape[0])+': ************')
    score = modelC.score(X_test_vectorized,y_test)
    #score = dicModel[typeML][labelData][label].score(X_test_vectorized,y_test)

    print(str(datetime.now())+" -     score : "+str(100*score)+" - score train : "+str(100*modelC.score(X_train_vectorized, y_train)))
    #print(str(datetime.now())+" -   score train : "+str(100*dicModel[typeML][labelData][label].score(X_train_vectorized, y_train)))
    
    return score, timer.interval, modelC


def testParam(param, criterion, n_estimators, tol, solver, loss, class_weight, typeML, \
              labelData, nomCols, X_train, X_test, y_train, y_test, etape):
    #for param in [valBase, 10*valBase, 100*valBase]:
    score, temps, model = apprendData(param, criterion, n_estimators, tol, solver, loss, class_weight,
                                      typeML, labelData, X_train, X_test, y_train, y_test)

    #if param == valBase:
    premier = chaine(param, score, temps, model)
    courant = premier
    #else:
    #    courant.setSuivant(chaine(param, score, temps))
    #    courant = courant.suivant
    
    oldValMax =-1
    oldScoreMax =-1
    
    if etape == 1:
        maxVal = 0
    elif etape == 2:
        maxVal = 2
    else:
        maxVal = 4
    
    #for i in range(4, (etape+1)*(etape+1)):
    for i in range(0, maxVal):
        print(i)
        maxMaillon, place = getMaillonScoreMax(premier)
        (nVP, nVS) = newValAutourduMax(maxMaillon)
        
        scoreC = maxMaillon.score
        if oldScoreMax < scoreC:
            oldScoreMax = scoreC
            print("nouveau Max !! : "+ str(100*scoreC))
        
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+" - max Score : "+str(100*maxMaillon.score)+" / val : "+str(maxMaillon.valeur))

        scoreP, tempsP, modelP = apprendData(nVP, criterion, n_estimators, tol, solver, loss, class_weight,
                                             typeML, labelData, X_train, X_test, y_train, y_test)
        scoreS, tempsS, modelS = apprendData(nVS, criterion, n_estimators, tol, solver, loss, class_weight,
                                             typeML, labelData, X_train, X_test, y_train, y_test)

        maxMaillon.placeAvant(chaine(nVP,scoreP, tempsP, modelP))
        maxMaillon.placeApres(chaine(nVS,scoreS, tempsS, modelS))

        if place == 1:
            premier = maxMaillon.precedent
    
    maxMaillon, place = getMaillonScoreMax(premier)
    print()
    print("Max  : "+str(100*maxMaillon.score))
    
    #dump(maxMaillon.model, labelData+'_'+nomCols+'_'+typeML+'_best_'+solver+'_'+criterion+'_'+loss+'_'+str(maxMaillon.valeur)+'_'+str(100*maxMaillon.score)+'.modele')
    
    return maxMaillon.valeur, 100*maxMaillon.score, maxMaillon.temps, maxMaillon.model

In [5]:
def transformXY(vect_V, X_test, y_test, label, nomCols, filtre, etape):
    if filtre != '':
        X_testFiltre = X_test[X_test[label+str(etape-1)] == filtre]
        y_testF = X_test[X_test[label+str(etape-1)] == filtre]
        y_testF = y_testF['Etape'+str(etape)]
    else:
        X_testFiltre = X_test
        y_testF = y_test
    
    nbrLignesFiltre = len(X_testFiltre)
    #print('nbrLignesFiltre : '+str(nbrLignesFiltre))
    if nbrLignesFiltre > 0:
        X_Te = X_testFiltre[nomCols]
        X_test_vectorized = vect_V.transform(X_Te)
        OK = True
    else:
        X_test_vectorized=''
        OK = False

    return X_test_vectorized, y_testF, OK
    

def testTout(label, nomCols, vect_V, X_train_vectorized, X_test, y_train, y_test, filtre, etape):
    paramBaseLR = 60
    paramBasePA = 0.5
    paramBaseSGD = 1e-8
    paramBaseLSVC = 40
    paramBaseRF = 500
    
    LogistiqueRegression_OK = True
    PassiveAggressive_OK = True
    SGD_OK = True
    LinearSVC_OK = True
    RandomForest_OK = False
    
    recap=[]
    recapModels={}
    loss = "-"
    tol = 1e-4
    class_weight = None
    criterion = "-"
    solver = "-"
    n_estimators = "-"
    
    if LogistiqueRegression_OK:
        print('= = = = = = = = = = = = = = = >> Logistic Regression')
        #for solver in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
        for solver in ['liblinear', 'sag']: #, 'saga']:
            print()
            print("===> solver : "+str(solver))
            #for class_weight in ['balanced', None]:
            #    print()
            #    print("===> class_weight : "+str(class_weight))
                #for tol in [1e-4]:#[1e-5, 1e-4, 1e-3]:
                #    print("===> tol : "+str(tol))
            if solver == 'liblinear':
                if etape == 1:
                    paramBaseLR = 125
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseLR = 40
                    elif filtre[1]=='corporate services':
                        paramBaseLR = 176
                    elif filtre[1]=='professional services':
                        paramBaseLR = 54
                    elif filtre[1]=='technology':
                        paramBaseLR = 97
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseLR = 132
                    elif filtre[1]=='general services':
                        paramBaseLR = 59
                    elif filtre[1]=='travel':
                        paramBaseLR = 71
                    elif filtre[1]=='legal':
                        paramBaseLR = 37
                    elif filtre[1]=='marcom':
                        paramBaseLR = 82
                    elif filtre[1]=='recruitment fees':
                        paramBaseLR = 21
            else:
                if etape == 1:
                    paramBaseLR = 60
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseLR = 19
                    elif filtre[1]=='corporate services':
                        paramBaseLR = 60
                    elif filtre[1]=='professional services':
                        paramBaseLR = 48
                    elif filtre[1]=='technology':
                        paramBaseLR = 30
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseLR = 31
                    elif filtre[1]=='general services':
                        paramBaseLR = 13
                    elif filtre[1]=='travel':
                        paramBaseLR = 27
                    elif filtre[1]=='legal':
                        paramBaseLR = 6
                    elif filtre[1]=='marcom':
                        paramBaseLR = 39
                    elif filtre[1]=='recruitment fees':
                        paramBaseLR = 18
                
            X_test_vectorized, y_testF, contenuOK = transformXY(vect_V, X_test, y_test, 'LogisticRegression_'+solver+'_',
                                                     nomCols, filtre[1], etape)
            if contenuOK:
                maxVal, maxScore, maxTemps, maxModel = testParam(paramBaseLR, criterion, n_estimators, tol, solver, loss,
                                                                 class_weight, 'LogisticRegression', label, nomCols,
                                                                 X_train_vectorized, X_test_vectorized, y_train, y_testF, etape)
                recap.append([maxScore, maxVal, maxTemps, nomCols, 'LogisticRegression', 
                              tol, class_weight, solver, loss, filtre[1], etape])
            
                if etape > 1:
                    #print('vecteur etape precedente')
                    #print(X_test['LogisticRegression_'+solver+'_'+str(etape-1)])
                    #print()
                    #print('filtre cherché')
                    #print(filtre[1])
                    #print(filtre)
                    #print()
                    recapModels['LogisticRegression_'+solver+'_'+filtre[1]+'_'+str(etape)]=maxModel
                    valeurs = maxModel.predict(vect_V.transform(X_test[nomCols]))
                    X_test['LogisticRegression_'+solver+'_'+str(etape)] = \
                        [b if a == filtre[1] else c \
                         for a,b,c in zip(X_test['LogisticRegression_'+solver+'_'+str(etape-1)], valeurs, \
                                        X_test['LogisticRegression_'+solver+'_'+str(etape)])]
                    #print('vecteur construit à cette étape')
                    #print(X_test['LogisticRegression_'+solver+'_'+str(etape)])
                else:
                    recapModels['LogisticRegression_'+solver+'_'+str(etape)]=maxModel
                    X_test['LogisticRegression_'+solver+'_'+str(2)] = '-'
                    X_test['LogisticRegression_'+solver+'_'+str(3)] = '-'
            
    if PassiveAggressive_OK:
        print()
        print()
        print('= = = = = = = = = = = = = = = >> Passive Aggressive')
        solver="-"
        #for class_weight in ['balanced', None]:
        #    print()
        #    print("===> class_weight : "+str(class_weight))
            #for tol in [1e-4]:#[1e-4, 1e-3, 1e-2]:
            #    print("===> tol : "+str(tol))
        
        if etape == 2:
            if filtre[1]=='banking services':
                paramBasePA = 0.27
            elif filtre[1]=='corporate services':
                paramBasePA = 0.33
            elif filtre[1]=='professional services':
                paramBasePA = 0.76
            elif filtre[1]=='technology':
                paramBasePA = 0.31
        elif etape == 3:
            if filtre[1]=='corporate real estate':
                paramBasePA = 0.2
            elif filtre[1]=='general services':
                paramBasePA = 0.41
            elif filtre[1]=='travel':
                paramBasePA = 0.57
            elif filtre[1]=='legal':
                paramBasePA = 0.24
            elif filtre[1]=='marcom':
                paramBasePA = 0.22
            elif filtre[1]=='recruitment fees':
                paramBasePA = 0.09
        
        X_test_vectorized, y_testF, contenuOK = transformXY(vect_V, X_test, y_test, 'PassiveAggressive_', 
                                                 nomCols, filtre[1], etape)
        if contenuOK:
            maxVal, maxScore, maxTemps, maxModel = testParam(paramBasePA, criterion, n_estimators, tol, solver, loss,
                                                             class_weight, 'PassiveAggressiveClassifier', label, nomCols,
                                                             X_train_vectorized, X_test_vectorized, y_train, y_testF, etape)
            recap.append([maxScore, maxVal, maxTemps, nomCols, 'PassiveAggressive', 
                          tol, class_weight, solver, loss, filtre[1], etape])
            #recapModels['PassiveAggressive'+'_'+filtre[1]+'_'+str(etape)]=maxModel

            if etape > 1:
                recapModels['PassiveAggressive_'+filtre[1]+'_'+str(etape)]=maxModel
                valeurs = maxModel.predict(vect_V.transform(X_test[nomCols]))
                X_test['PassiveAggressive_'+str(etape)] = \
                    [b if a == filtre[1] else c \
                     for a,b,c in zip(X_test['PassiveAggressive_'+str(etape-1)], valeurs, \
                                    X_test['PassiveAggressive_'+str(etape)])]
            else:
                recapModels['PassiveAggressive'+'_'+str(etape)]=maxModel
                X_test['PassiveAggressive_'+str(2)] = '-'
                X_test['PassiveAggressive_'+str(3)] = '-'
                
    if SGD_OK:
        print()
        print()
        print('= = = = = = = = = = = = = = = >> SGD')
        for loss in ['hinge', 'log', 'modified_huber']:#, 'squared_hinge', 'perceptron']:
            print()
            print("===> loss : "+str(loss))
            #for class_weight in ['balanced', None]:
            #    print()
            #    print("===> class_weight : "+str(class_weight))
                #for tol in [1e-4]:#[1e-4, 1e-3, 1e-2]:
                #    print("===> tol : "+str(tol))
            if loss=='hinge':
                if etape == 1:
                    paramBaseSGD = 2e-8
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseSGD = 6.3e-9
                    elif filtre[1]=='corporate services':
                        paramBaseSGD = 2.8e-8
                    elif filtre[1]=='professional services':
                        paramBaseSGD = 1.5e-8
                    elif filtre[1]=='technology':
                        paramBaseSGD = 1.8e-8
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseSGD = 2e-8
                    elif filtre[1]=='general services':
                        paramBaseSGD = 1.6e-8
                    elif filtre[1]=='travel':
                        paramBaseSGD = 5.4e-8
                    elif filtre[1]=='legal':
                        paramBaseSGD = 2e-9
                    elif filtre[1]=='marcom':
                        paramBaseSGD = 1.6e-8
                    elif filtre[1]=='recruitment fees':
                        paramBaseSGD = 7e-9
            elif loss=='log':
                if etape == 1:
                    paramBaseSGD = 1e-8
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseSGD = 6.7e-9
                    elif filtre[1]=='corporate services':
                        paramBaseSGD = 8.4e-9
                    elif filtre[1]=='professional services':
                        paramBaseSGD = 1e-8
                    elif filtre[1]=='technology':
                        paramBaseSGD = 8e-9
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseSGD = 1.4e-8
                    elif filtre[1]=='general services':
                        paramBaseSGD = 1e-8
                    elif filtre[1]=='travel':
                        paramBaseSGD = 1.1e-8
                    elif filtre[1]=='legal':
                        paramBaseSGD = 2.1e-9
                    elif filtre[1]=='marcom':
                        paramBaseSGD = 1.1e-8
                    elif filtre[1]=='recruitment fees':
                        paramBaseSGD = 5.5e-9
            elif loss=='modified_huber':
                if etape == 1:
                    paramBaseSGD = 1e-7
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseSGD = 4.5e-8
                    elif filtre[1]=='corporate services':
                        paramBaseSGD = 1.75e-7
                    elif filtre[1]=='professional services':
                        paramBaseSGD = 1.46e-7
                    elif filtre[1]=='technology':
                        paramBaseSGD = 9.2e-8
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseSGD = 8.7e-8
                    elif filtre[1]=='general services':
                        paramBaseSGD = 1.1e-7
                    elif filtre[1]=='travel':
                        paramBaseSGD = 1.5e-7
                    elif filtre[1]=='legal':
                        paramBaseSGD = 1e-8
                    elif filtre[1]=='marcom':
                        paramBaseSGD = 1.5e-7
                    elif filtre[1]=='recruitment fees':
                        paramBaseSGD = 6.3e-8
        
            X_test_vectorized, y_testF, contenuOK = transformXY(vect_V, X_test, y_test, 'SGD_'+loss+'_', 
                                                     nomCols, filtre[1], etape)
            if contenuOK:
                maxVal, maxScore, maxTemps, maxModel = testParam(paramBaseSGD, criterion, n_estimators, tol, solver,
                                                                 loss, class_weight, 'SGDClassifier', label, nomCols,
                                                                 X_train_vectorized, X_test_vectorized, y_train, y_testF, etape)
                recap.append([maxScore, maxVal, maxTemps, nomCols, 'SGD', tol, class_weight, 
                              solver, loss, filtre[1], etape])
                #recapModels['SGD_'+loss+'_'+filtre[1]+'_'+str(etape)]=maxModel

                if etape > 1:
                    recapModels['SGD_'+loss+'_'+filtre[1]+'_'+str(etape)]=maxModel
                    valeurs = maxModel.predict(vect_V.transform(X_test[nomCols]))
                    X_test['SGD_'+loss+'_'+str(etape)] = \
                        [b if a == filtre[1] else c \
                         for a,b,c in zip(X_test['SGD_'+loss+'_'+str(etape-1)], valeurs, \
                                        X_test['SGD_'+loss+'_'+str(etape)])]
                else:
                    recapModels['SGD_'+loss+'_'+str(etape)]=maxModel
                    X_test['SGD_'+loss+'_'+str(2)] = '-'
                    X_test['SGD_'+loss+'_'+str(3)] = '-'
             
    if LinearSVC_OK:
        print()
        print()
        print('= = = = = = = = = = = = = = = >> Linear SVC')
        for loss in ['hinge', 'squared_hinge']:
            print()
            print("===> loss : "+str(loss))
            #for class_weight in ['balanced', None]:
            #    print()
            #    print("===> class_weight : "+str(class_weight))
                #for tol in [1e-4]:#[1e-5, 1e-4, 1e-3]:
                #    print("===> tol : "+str(tol))
            if loss=='hinge':
                if etape == 1:
                    paramBaseLSVC = 40
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseLSVC = 21.7
                    elif filtre[1]=='corporate services':
                        paramBaseLSVC = 32
                    elif filtre[1]=='professional services':
                        paramBaseLSVC = 46.2
                    elif filtre[1]=='technology':
                        paramBaseLSVC = 18
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseLSVC = 36.3
                    elif filtre[1]=='general services':
                        paramBaseLSVC = 23.3
                    elif filtre[1]=='travel':
                        paramBaseLSVC = 52
                    elif filtre[1]=='legal':
                        paramBaseLSVC = 6.6
                    elif filtre[1]=='marcom':
                        paramBaseLSVC = 35
                    elif filtre[1]=='recruitment fees':
                        paramBaseLSVC = 5.4
            elif loss=='squared_hinge':
                if etape == 1:
                    paramBaseLSVC = 7.5
                elif etape == 2:
                    if filtre[1]=='banking services':
                        paramBaseLSVC = 2.4
                    elif filtre[1]=='corporate services':
                        paramBaseLSVC = 3.2
                    elif filtre[1]=='professional services':
                        paramBaseLSVC = 6.5
                    elif filtre[1]=='technology':
                        paramBaseLSVC = 5.5
                elif etape == 3:
                    if filtre[1]=='corporate real estate':
                        paramBaseLSVC = 3.4
                    elif filtre[1]=='general services':
                        paramBaseLSVC = 5.9
                    elif filtre[1]=='travel':
                        paramBaseLSVC = 5.3
                    elif filtre[1]=='legal':
                        paramBaseLSVC = 1.2
                    elif filtre[1]=='marcom':
                        paramBaseLSVC = 5.1
                    elif filtre[1]=='recruitment fees':
                        paramBaseLSVC = 1.9
        
            X_test_vectorized, y_testF, contenuOK = transformXY(vect_V, X_test, y_test, 'LinearSVC_'+loss+'_', 
                                                     nomCols, filtre[1], etape)
            if contenuOK:
                maxVal, maxScore, maxTemps, maxModel = testParam(paramBaseLSVC, criterion, n_estimators, tol, solver,
                                                                 loss, class_weight, 'LinearSVC', label, nomCols,
                                                                 X_train_vectorized, X_test_vectorized, y_train, y_testF, etape)
                recap.append([maxScore, maxVal, maxTemps, nomCols, 'LinearSVC', tol, class_weight, 
                              solver, loss, filtre[1], etape])
                #recapModels['LinearSVC_'+loss+'_'+filtre[1]+'_'+str(etape)]=maxModel

                if etape > 1:
                    recapModels['LinearSVC_'+loss+'_'+filtre[1]+'_'+str(etape)]=maxModel
                    valeurs = maxModel.predict(vect_V.transform(X_test[nomCols]))
                    X_test['LinearSVC_'+loss+'_'+str(etape)] = \
                        [b if a == filtre[1] else c \
                         for a,b,c in zip(X_test['LinearSVC_'+loss+'_'+str(etape-1)], valeurs, \
                                        X_test['LinearSVC_'+loss+'_'+str(etape)])]
                else:
                    recapModels['LinearSVC_'+loss+'_'+str(etape)]=maxModel
                    X_test['LinearSVC_'+loss+'_'+str(2)] = '-'
                    X_test['LinearSVC_'+loss+'_'+str(3)] = '-'
               
    if RandomForest_OK:
        print()
        print()
        print('= = = = = = = = = = = = = = = >> Random Forest')
        for criterion in ['gini', 'entropy']:
            print()
            print("===> criterion : "+str(criterion))
            for n_estimators in [100]: #[50, 100, 150, 200]:
                print()
                print("===> n_estimators : "+str(n_estimators))
                #for tol in [1e-4]:#[1e-5, 1e-4, 1e-3]:
                #    print("===> tol : "+str(tol))
        
                X_test_vectorized, y_testF, contenuOK = transformXY(vect_V, X_test, y_test, 'RandomForest_'+criterion+'_', 
                                                         nomCols, filtre[1], etape)
                if contenuOK:
                    maxVal, maxScore, maxTemps, maxModel = testParam(paramBaseRF, criterion, n_estimators, tol,
                                                                     solver, loss, class_weight, 'RandomForest',
                                                                     label, nomCols, X_train_vectorized, X_test_vectorized,
                                                                     y_train, y_testF, etape)
                    recap.append([maxScore, maxVal, maxTemps, nomCols, 'RandomForest',
                                  tol, class_weight, solver, loss, filtre[1], etape])
                    #recapModels['RandomForest_'+criterion+'_'+filtre[1]+'_'+str(etape)]=maxModel

                    if etape > 1:
                        recapModels['RandomForest_'+loss+'_'+filtre[1]+'_'+str(etape)]=maxModel
                        valeurs = maxModel.predict(vect_V.transform(X_test[nomCols]))
                        X_test['RandomForest_'+loss+'_'+str(etape)] = \
                            [b if a == filtre[1] else c \
                             for a,b,c in zip(X_test['RandomForest_'+loss+'_'+str(etape-1)], valeurs, \
                                            X_test['RandomForest_'+loss+'_'+str(etape)])]
                    else:
                        recapModels['RandomForest_'+loss+'_'+str(etape)]=maxModel
                        X_test['RandomForest_'+loss+'_'+str(2)] = '-'
                        X_test['RandomForest_'+loss+'_'+str(3)] = '-'
               
    #dfRT = pd.DataFrame(recap, columns =['score', 'valeur', 'temps', 'nomCols', 'typeML', 'tol',
    #                                     'class_weight', 'solver', 'loss', 'filtre', 'etape'])
    #dfRT.to_excel(nomCols+'_'+filtre[1]+'_'+str(etape)+' - stats réduites.xlsx')
    
    return recap, recapModels, X_test, contenuOK

In [6]:
def testCol(X_Train, X_Test, y_Train, y_Test, vectorizer, filtre, etape):
    recap=[]
    
    for nomCols in ["AcléEt"]: #, "AcléEtSansE", "AcléMinE", "AcléMin"]:
        X_Tr=X_Train[nomCols]
        X_Te=X_Test[nomCols]

        with Timer() as timer:
            vect_V = vectorizer.fit(X_Tr)
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement du fitting de '+nomCols+'+'+filtre[1]+': '+str(timer.interval))
        
        with Timer() as timer:
            X_TrV = vect_V.transform(X_Tr)
            #X_TeV = vect_V.transform(X_Te)
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement du transform : '+str(timer.interval))

        print()
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' !!! GO !!!')

        with Timer() as timerCol:
            #minirecap, recapModels = testTout('1821', nomCols, X_TrV, X_TeV, y_Train, y_Test, filtre, etape)
            minirecap, recapModels, retourXTest, contenuOK = testTout('1821', nomCols, vect_V, X_TrV, X_Test, \
                                                                      y_Train, y_Test, filtre, etape)
        print()
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement de '+nomCols+': '+str(timerCol.interval))
        print()
        
        recap = recap + minirecap
    
    
    if etape == 1:
        dfR = X_Test.copy()
        X_to_predict = vect_V.transform(dfR["AcléEt"])
        for label,model in recapModels.items():
            dfR[label] = model.predict(X_to_predict)
            #print(label)
    else:
        dfR = retourXTest
    
    #dfRT = pd.DataFrame(recap, columns =['score', 'valeur', 'temps', 'nomCols', 'typeML', 'tol',
    #                                     'class_weight', 'solver', 'loss', 'filtre', 'etape'])
    #dfRT.to_excel(str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+filtre[1]+'_'+str(etape)+' - Stats globales.xlsx')
    
    #dfR.to_excel(str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+filtre[1]+'_'+str(etape)+' - sorties de Prediction.xlsx')
    
    return recap, dfR

def apprentissage(df, maxL, testSize, typeApprentissage, num_essai):
    print("---------------------------------------------------------------")
    print("-----------------------------DEBUT-----------------------------")
    print("---------------------------------------------------------------")
    print()
    print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - nombre de lignes : '+str(maxL))
    print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - taille du test : '+str(100*testSize)+'%')
    print()
    
    df['Etape1']=df['Family Stream']
    df['Etape2']=df['Category Stream']
    df['Etape3']=df['Sub Category Stream']
    
    with Timer() as timer:
        if maxL > 0:
            X_Train, X_Test, y_Train, y_Test = train_test_split(df[:maxL], df["Acible"][:maxL], test_size = testSize)
        else:
            X_Train, X_Test, y_Train, y_Test = train_test_split(df, df["Acible"], test_size = testSize)
    print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement du TTS : '+str(timer.interval))
    
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3),
                         min_df = 3, stop_words = 'english', sublinear_tf=True)
    recapTout=[]
    if typeApprentissage == 'tout':
        with Timer() as timerTout:
            if False:
                recap, dfR = testCol(X_Train, X_Test, y_Train, y_Test, vectorizer, ['', ''], 1)
            else:
                recap, dfR = testCol(X_Train, X_Test, X_Train["Family Stream"], X_Test["Family Stream"], \
                                     vectorizer, ['', ''], 1)
        print()
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement : '+str(timerTout.interval))
        recapTout = recapTout + recap
        
        dfRT = pd.DataFrame(recap, columns =['score', 'valeur', 'temps', 'nomCols', 'typeML', 'tol',
                                             'class_weight', 'solver', 'loss', 'filtre', 'etape'])
        dfRT.to_excel(str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+str(num_essai)+' - Stats globales.xlsx')

        dfR.to_excel(str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+str(num_essai)+' - sorties de Prediction.xlsx')
        dump(dfR, str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+str(num_essai)+' - sorties de Prediction.dump')
        
    else:
        dfR = None
        print()
        print('===> Etape 1')
        with Timer() as timerTout:
            recap, dfRFamily = testCol(X_Train, X_Test, X_Train["Family Stream"], X_Test["Family Stream"],
                                       vectorizer, ['', ''], 1)
        print()
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement : '+str(timerTout.interval))
        recapTout = recapTout + recap
        
        X_TestFS = dfRFamily
        for familyStream in df.groupby(['Family Stream']).count().transpose().columns:
            print()
            print('===> Etape 2')
            print()
            print(familyStream)
            
            X_TrainFS = X_Train[X_Train['Family Stream'] == familyStream]
            print(X_TrainFS.shape)
            nbrCat = len(X_TrainFS.groupby(['Category Stream']))
            print(nbrCat)
            if nbrCat > 1 and X_TrainFS.shape[0] > 0:
                with Timer() as timerTout:
                    recap, X_TestFS = testCol(X_TrainFS, X_TestFS, X_TrainFS["Category Stream"], X_TestFS,
                                                 vectorizer, ['', familyStream], 2)
                print(len(X_TestFS))
                print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement : '+str(timerTout.interval))
                recapTout = recapTout + recap

                X_TestCS = X_TestFS
                for categoryStream in X_TrainFS.groupby(['Category Stream']).count().transpose().columns:
                    print()
                    print('===> Etape 3')
                    print()
                    print(categoryStream)

                    X_TrainCS = X_TrainFS[X_TrainFS['Category Stream'] == categoryStream]
                    print(X_TrainCS.shape)
                    nbrSousCat = len(X_TrainCS.groupby(['Sub Category Stream']))
                    print(nbrSousCat)
                    if nbrSousCat > 1 and X_TrainCS.shape[0] > 0:
                        with Timer() as timerTout:
                            recap, X_TestCS = testCol(X_TrainCS, X_TestCS, X_TrainCS["Sub Category Stream"], X_TestCS,
                                                     vectorizer, [familyStream, categoryStream], 3)
                        print(len(X_TestCS))
                        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement : '+str(timerTout.interval))
                        recapTout = recapTout + recap
                        
        dfR = X_TestCS
    
    return recapTout, dfR

# APPRENTISSAGE

In [ ]:
dfToLearn = df1821.copy()

apprentissage(dfToLearn, 1000, 0, '', 0)

# RESTITUTION

## TESTS Maximiser score d'apprentissage

In [60]:
dfToLearn = df1821.copy()

print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' !!!!! START !!!!!')

retoursAll1=[]
#'tout' en dernier argument pour apprendre sur la concatenation de Family+Category+Sub Category
#n'importe quoi d'autre pour apprendre par étape sur Family puis Category puis Sub Category
with Timer() as timerComplet:
    for i in range(1,5):
        with Timer() as timerTout:
            retoursAll1.append(apprentissage(dfToLearn, -1, 0.005, '', i))
            #recapTout2, dfRetour2 = apprentissage(dfToLearn, 100, 0.05, 'tout')
            dump(retoursAll1, str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+'retoursAll1.dump')
        print()
        print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement global : '+str(timerTout.interval))

print()
print(str(datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))+' - temps de traitement complet : '+str(timerComplet.interval))
dump(retoursAll1, str(datetime.now().strftime("%Y.%m.%d_%H.%M.%S"))+'_'+'Fin retoursAll1.dump')

2021-06-17_16:14:50 !!!!! START !!!!!
---------------------------------------------------------------
-----------------------------DEBUT-----------------------------
---------------------------------------------------------------

2021-06-17_16:14:50 - nombre de lignes : -1
2021-06-17_16:14:50 - taille du test : 0.5%

2021-06-17_16:14:56 - temps de traitement du TTS : 5.9462730884552

===> Etape 1
2021-06-17_16:15:54 - temps de traitement du fitting de AcléEt+: 57.51562309265137
2021-06-17_16:16:49 - temps de traitement du transform : 55.15053415298462

2021-06-17_16:16:49 !!! GO !!!
= = = = = = = = = = = = = = = >> Logistic Regression

===> solver : liblinear
2021-06-17 17:28:45.884186 - valeur : 125 - fit en : 4316.420648097992
2021-06-17 17:28:45.904328 -     score : 96.05301485581124 - score train : 97.93142066602985

Max  : 96.05301485581124

===> solver : sag


<ipython-input-55-ef25b6a5550b>:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['LogisticRegression_'+solver+'_'+str(2)] = '-'
<ipython-input-55-ef25b6a5550b>:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['LogisticRegression_'+solver+'_'+str(3)] = '-'


2021-06-17 17:38:18.094885 - valeur : 60 - fit en : 567.5789940357208
2021-06-17 17:38:18.189679 -     score : 96.09670841829305 - score train : 97.98170778571978

Max  : 96.09670841829305

===> solver : saga


<ipython-input-55-ef25b6a5550b>:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['LogisticRegression_'+solver+'_'+str(2)] = '-'
<ipython-input-55-ef25b6a5550b>:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['LogisticRegression_'+solver+'_'+str(3)] = '-'


2021-06-17 17:57:29.950193 - valeur : 60 - fit en : 1146.9929914474487
2021-06-17 17:57:30.002888 -     score : 96.09670841829305 - score train : 97.98148819131065

Max  : 96.09670841829305


= = = = = = = = = = = = = = = >> Passive Aggressive


...

2021-06-22 23:41:25.218898 - valeur : 40 - fit en : 3.8689863681793213
2021-06-22 23:41:25.218898 -     score : 91.8918918918919 - score train : 98.61245173745174
-3
nouveau Max !! : 91.8918918918919
2021-06-22_23:41:25 - max Score : 91.8918918918919 / val : 40
2021-06-22 23:41:28.794975 - valeur : 22.493653007613965 - fit en : 3.5346837043762207
2021-06-22 23:41:28.797142 -     score : 91.8918918918919 - score train : 98.55212355212355
2021-06-22 23:41:32.349714 - valeur : 71.13117640155691 - fit en : 3.5211684703826904
2021-06-22 23:41:32.349714 -     score : 91.8918918918919 - score train : 98.76930501930502
-2
2021-06-22_23:41:32 - max Score : 91.8918918918919 / val : 22.493653007613965
2021-06-22 23:41:35.874253 - valeur : 12.649110640673518 - fit en : 3.4836668968200684
2021-06-22 23:41:35.875321 -     score : 94.5945945945946 - score train : 98.33494208494209
2021-06-22 23:41:39.863577 - valeur : 25.975265263048456 - fit en : 3.9678540229797363
2021-06-22 23:41:39.863577 -     s

2021-06-23 00:23:57.564433 - valeur : 0.28117066259517454 - fit en : 28.09174156188965
2021-06-23 00:23:57.585134 -     score : 92.05219454329774 - score train : 99.19667590027701
2021-06-23 00:24:21.687791 - valeur : 0.8891397050194614 - fit en : 22.881296396255493
2021-06-23 00:24:21.705988 -     score : 91.57769869513642 - score train : 99.27736962543659
-2
nouveau Max !! : 92.05219454329774
2021-06-23_00:24:22 - max Score : 92.05219454329774 / val : 0.28117066259517454
2021-06-23 00:24:51.077240 - valeur : 0.15811388300841897 - fit en : 28.24573826789856
2021-06-23 00:24:51.100474 -     score : 92.17081850533808 - score train : 99.09550764783813
2021-06-23 00:25:16.873200 - valeur : 0.32469081578810566 - fit en : 24.348133325576782
2021-06-23 00:25:16.896054 -     score : 92.17081850533808 - score train : 99.20089124412863

Max  : 92.17081850533808


= = = = = = = = = = = = = = = >> SGD

===> loss : hinge
2021-06-23 00:26:00.904454 - valeur : 2e-08 - fit en : 41.874770402908325
202

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


2021-06-23 00:40:55.813941 - valeur : 40 - fit en : 129.99120950698853
2021-06-23 00:40:55.822072 -     score : 91.68646080760095 - score train : 99.23280741900517
-3
nouveau Max !! : 91.68646080760095
2021-06-23_00:40:56 - max Score : 91.68646080760095 / val : 40
2021-06-23 00:43:06.954067 - valeur : 22.493653007613965 - fit en : 130.0379490852356
2021-06-23 00:43:06.954067 -     score : 91.68646080760095 - score train : 99.15933999759123
2021-06-23 00:45:27.249561 - valeur : 71.13117640155691 - fit en : 139.0103988647461
2021-06-23 00:45:27.249561 -     score : 91.33016627078385 - score train : 99.28339154522462
-2
2021-06-23_00:45:28 - max Score : 91.68646080760095 / val : 22.493653007613965
2021-06-23 00:47:39.191773 - valeur : 12.649110640673518 - fit en : 130.85252928733826
2021-06-23 00:47:39.193888 -     score : 91.68646080760095 - score train : 99.06600024087679
2021-06-23 00:49:48.867022 - valeur : 25.975265263048456 - fit en : 128.56883430480957
2021-06-23 00:49:48.869240 - 

['2021.06.23_00.55.48_Fin retoursAll1.dump']

In [7]:
retoursAll = load('2021.06.23_00.55.48_Fin retoursAll1.dump')

In [8]:
retoursAll[0][1]

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Cd_Cat,Catégorie_de_dépenses,Cd_Fam,...,LinearSVC_squared_hinge_3,LogisticRegression_liblinear_1,LogisticRegression_sag_1,LogisticRegression_saga_1,PassiveAggressive_1,SGD_hinge_1,SGD_log_1,SGD_modified_huber_1,LinearSVC_hinge_1,LinearSVC_squared_hinge_1
581912,US001,bank of the west,US,BOW.628055,employee welfare,amedi,Médical,PREV,Prévoyance collaborateurs et leurs biens,ASSU,...,-,corporate insurance,corporate insurance,corporate insurance,corporate insurance,corporate insurance,corporate insurance,corporate insurance,corporate insurance,corporate insurance
1215707,US001,bank of the west,-,-,coll and repo - appraisal-environmental,hrsrv0,z-Prestations externalisées liées à la personn...,HRSRV,Prestations externalisées liées à la personne,SPRO,...,-,professional services,professional services,professional services,professional services,professional services,professional services,professional services,professional services,professional services
1101243,FR36179,domofinance,-,-,traning - performance with convention,fpctrn,Prestations,FPC,Formation,SPRO,...,-,professional services,professional services,professional services,professional services,professional services,professional services,professional services,professional services,professional services
404357,ES018,uci espana,ES,UCI.62986,comunicaciœn interna,edit0,z-Edition - autres,EDIT,Edition,C&MK,...,advertising (incl purchase of space & creation...,professional services,professional services,professional services,professional services,professional services,professional services,professional services,professional services,professional services
440299,US001,bank of the west,US,BOW.662189,membership club dues-individual,abonn,Abonnements professionnels,COTIS,Cotisations et abonnements,INFO,...,air,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311144,HK011,bnpp hong kong branch,-,-,entertaining - client (restaurants),resto,Restaurants,RESTO,Restaurants,VOYAG,...,restaurants,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services
140953,US001,bank of the west,US,BOW.671107,postage and freight exp,timbr,Affranchissement postal,POSTE,Affranchissement et courrier,C&MG,...,postage,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services
775780,BR22888,cardif do brasil vida e previdencia sa,-,-,licencas de uso de software,sftw0,z-Logiciels - autres,SOFTW,Logiciels,TECH,...,-,technology,technology,technology,technology,technology,technology,technology,technology,technology
1192087,US001,bank of the west,-,-,rent office space,loyer,Loyer,LOYER,Coûts d'occupation,IMMO,...,costs of occupancy,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services,corporate services


In [9]:
dfCompareVal0 = pd.DataFrame(retoursAll[0][0], columns =['score', 'valeur', 'temps', 'nomCols', 'typeML', 'tol',
                                             'class_weight', 'solver', 'loss', 'filtre', 'etape'])
dfCompareVal1 = pd.DataFrame(retoursAll[1][0], columns =['score', 'valeur', 'temps', 'nomCols', 'typeML', 'tol',
                                             'class_weight', 'solver', 'loss', 'filtre', 'etape'])
dfCompareVal2 = pd.DataFrame(retoursAll[2][0], columns =['score', 'valeur', 'temps', 'nomCols', 'typeML', 'tol',
                                             'class_weight', 'solver', 'loss', 'filtre', 'etape'])
dfCompareVal3 = pd.DataFrame(retoursAll[3][0], columns =['score', 'valeur', 'temps', 'nomCols', 'typeML', 'tol',
                                             'class_weight', 'solver', 'loss', 'filtre', 'etape'])

In [10]:
dfCompareVal0

,score,valeur,temps,nomCols,typeML,tol,class_weight,solver,loss,filtre,etape
0,96.053015,1.250000e+02,4316.420648,AcléEt,LogisticRegression,0.0001,None,liblinear,-,,1
1,96.096708,6.000000e+01,567.578994,AcléEt,LogisticRegression,0.0001,None,sag,-,,1
2,96.096708,6.000000e+01,1146.992991,AcléEt,LogisticRegression,0.0001,None,saga,-,,1
3,95.616079,5.000000e-01,142.851047,AcléEt,PassiveAggressive,0.0001,None,-,-,,1
4,95.834547,2.000000e-08,196.248766,AcléEt,SGD,0.0001,None,-,hinge,,1
...,...,...,...,...,...,...,...,...,...,...,...
94,92.079208,1.124683e-08,24.121517,AcléEt,SGD,0.0001,None,-,hinge,technology,2
95,92.670807,1.000000e-08,27.768089,AcléEt,SGD,0.0001,None,-,log,technology,2
96,89.561091,1.778279e-07,20.249422,AcléEt,SGD,0.0001,None,-,modified_huber,technology,2
97,91.449814,1.264911e+01,57.170998,AcléEt,LinearSVC,0.0001,None,-,hinge,technology,2


In [73]:
dfCompareVal3.to_excel('CompareVal3.xlsx')

In [74]:
def agregeSortiesParNiveau(df):
    labels = ['LogisticRegression_liblinear_','LogisticRegression_sag_','LogisticRegression_saga_',\
                     'PassiveAggressive_','SGD_hinge_','SGD_log_','SGD_modified_huber_','LinearSVC_hinge_',\
                     'LinearSVC_squared_hinge_']
    
    for elem in labels:
        courant = None
        for i in range(1,4):
            if courant is None:
                courant = df[elem+str(i)]
            else:
                courant = courant + " ; " + df[elem+str(i)]
        
        df[elem + '123'] = courant

In [77]:
def traiteRecursifDf(df, dico, nbLignes, nbrElemTotal, nbrParmiTotal, numCourant, numElem, numInitial, \
                     tableauVal, complementLabel):
    
    if numCourant == nbrParmiTotal:
        label = ''
        
        for i in range(0, len(tableauVal) - 1):
            label = label+str(tableauVal[i])
        
        labelPrec = label
        labelBase = str(tableauVal[0]) + str(tableauVal[len(tableauVal) - 1])
        label = label+str(tableauVal[len(tableauVal) - 1])
        
        #print(label)
        
        dfTemp = pd.DataFrame()
        if complementLabel == '':
            if nbrParmiTotal == 2:
                df[label] = [1 if a==b else 0 for a,b in zip(df[tableauVal[0]],df[tableauVal[1]])]
            else:
                df[label] = [a*b for a,b in zip(df[labelBase],df[labelPrec])]
            
            dico[label] = {}
            dico[label]['%'] = 100*sum(df[label])/nbLignes
        else:
            df[label+complementLabel]=[a*b for a,b in zip(df[str(numInitial)+complementLabel],df[label])]
            if sum(df[label]) > 0:
                dico[label]['%'+complementLabel] = 100*sum(df[label+complementLabel])/sum(df[label])
            else:
                dico[label]['%'+complementLabel] = 0

        dico[label]['nbr'+complementLabel] = sum(df[label+complementLabel])

    else:
        for i in range(numElem+1, nbrElemTotal + 1):
            tableauVal.append(i)
            df, dico = traiteRecursifDf(df, dico, nbLignes, nbrElemTotal, nbrParmiTotal, numCourant+1, i, \
                                        numInitial, tableauVal, complementLabel)
            tableauVal.pop()
            
    return df, dico

def traiteDFRetourRec(dfTravail, champCible, ext):
    tableauLabels = ['LogisticRegression_liblinear_'+ext,'LogisticRegression_sag_'+ext,'LogisticRegression_saga_'+ext,\
                     'PassiveAggressive_'+ext,'SGD_hinge_'+ext,'SGD_log_'+ext,'SGD_modified_huber_'+ext,'LinearSVC_hinge_'+ext,\
                     'LinearSVC_squared_hinge_'+ext]
    nbTotalElem = len(tableauLabels)
    for i,label in zip([x for x in range(1, nbTotalElem + 1)],tableauLabels):
        dfTravail[i]=dfTravail[label]
    
    nbLignes = dfTravail.shape[0]
    
    dicoTemp={}
    dicoTemp['nbL'] = {}
    dicoTemp['nbL']['nbr'] = nbLignes
    dicoTemp['nbL']['%'] = 0
    dicoTemp['nbL']['nbr OK'] = 0
    dicoTemp['nbL']['% OK'] = 0
    
    #si chacun est pareil que cible, résultat attendu
    for i in range(1, nbTotalElem + 1):
        dfTravail[str(i)+' OK'] = [1 if a==b else 0 for a,b in zip(dfTravail[i],dfTravail[champCible])]
        dicoTemp[str(i)] = {}
        dicoTemp[str(i)]['nbr'] = nbLignes
        dicoTemp[str(i)]['%'] = 100
        dicoTemp[str(i)]['nbr OK'] = sum(dfTravail[str(i)+' OK'])
        dicoTemp[str(i)]['% OK'] = 100*sum(dfTravail[str(i)+' OK'])/nbLignes
    
    #lance le travail récursif
    #considère nbrElemPrisEnCompte parmi le nombre total d'éléments
    #soit 2 parmi n, puis 3 parmi n,...
    for nbrElemPrisEnCompte in range(2, len(tableauLabels) + 1):
        for i in range(1, len(tableauLabels) + 1):
            dfTravail, dicoTemp = traiteRecursifDf(dfTravail, dicoTemp, nbLignes, len(tableauLabels), \
                                                   nbrElemPrisEnCompte, 1, i, i, [i], '')
            dfTravail, dicoTemp = traiteRecursifDf(dfTravail, dicoTemp, nbLignes, len(tableauLabels), \
                                                   nbrElemPrisEnCompte, 1, i, i, [i], ' OK')
    
    dfResultatsT = pd.DataFrame.from_dict(dicoTemp)
    
    return dfTravail, dfResultatsT

In [138]:
dfRetourO = retoursAll[3][1].copy()

In [139]:
agregeSortiesParNiveau(dfRetourO)

In [145]:
dfRetour = dfRetourO[dfRetourO['Family Stream']=='banking services']

In [146]:
dfRetour

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Cd_Cat,Catégorie_de_dépenses,Cd_Fam,...,LinearSVC_squared_hinge_1,LogisticRegression_liblinear_123,LogisticRegression_sag_123,LogisticRegression_saga_123,PassiveAggressive_123,SGD_hinge_123,SGD_log_123,SGD_modified_huber_123,LinearSVC_hinge_123,LinearSVC_squared_hinge_123
475720,HU003,magyar cetelem bank,HU,CETMAGYAR.82110000,postai pénzforg. díj_paie,paie0,z-Moyens de paiement - autres,PAIE,Moyens de paiement,BANC,...,banking services,banking services ; banking software & services...,banking services ; banking software & services...,banking services ; banking software & services...,banking services ; banking software & services...,banking services ; banking software & services...,banking services ; banking software & services...,banking services ; banking software & services...,banking services ; banking software & services...,banking services ; banking software & services...
896038,FR8051,bddf - region grand est,-,-,checks subcontratcting fees,tchq,Traitement de chèques,TRVAL,Traitement des valeurs,BANC,...,banking services,banking services ; cheques ; -,banking services ; cheques ; -,banking services ; cheques ; -,banking services ; cheques ; -,banking services ; cheques ; -,banking services ; cheques ; -,banking services ; cheques ; -,banking services ; cheques ; -,banking services ; cheques ; -
1109843,US001,bank of the west,-,-,maint - atms only,gautm,Gestion d'automates,ELV,Prestations assurées par ELV,BANC,...,banking services,banking services ; atm ; -,banking services ; atm ; -,banking services ; atm ; -,banking services ; atm ; -,banking services ; atm ; -,banking services ; atm ; -,banking services ; atm ; -,banking services ; atm ; -,banking services ; atm ; -
743195,RU41332,arval ooo,RU,ARVAL_RU.65232001,employee car - fuel,carbu,Carburant,AUTO,Voitures,SPRO,...,banking services,corporate services ; general services ; car fleet,corporate services ; general services ; car fleet,corporate services ; general services ; car fleet,banking services ; banking software & services...,banking services ; banking software & services...,corporate services ; general services ; car fleet,banking services ; banking software & services...,banking services ; banking software & services...,banking services ; banking software & services...
975925,LU45555,banque generale du luxembourg,-,-,issuing token debet card,carte,Cartes bancaires,PAIE,Moyens de paiement,BANC,...,banking services,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...
870912,FR08081,bddf - pilotage especes & automates,-,-,funds transportation - atm maintenance,elv0,z-Prestations assurées par ELV - autres,ELV,Prestations assurées par ELV,BANC,...,banking services,banking services ; cash transport ; -,banking services ; cash transport ; -,banking services ; cash transport ; -,banking services ; cash transport ; -,banking services ; cash transport ; -,banking services ; cash transport ; -,banking services ; cash transport ; -,banking services ; cash transport ; -,banking services ; cash transport ; -
1083171,BE13394,bnpp fortis,-,-,issuing linked services credit card,paie0,z-Moyens de paiement - autres,PAIE,Moyens de paiement,BANC,...,banking services,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking services ; credit cards (incl processi...,banking s

In [147]:
with Timer() as timerTout:
    dfCompletRec, dfResultatsTRec = traiteDFRetourRec(dfRetour, 'Acible', '123')
print()
print(str(datetime.now())+' - temps de generation des possibles : '+str(timerTout.interval))

<ipython-input-77-406e00043759>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTravail[i]=dfTravail[label]
<ipython-input-77-406e00043759>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTravail[str(i)+' OK'] = [1 if a==b else 0 for a,b in zip(dfTravail[i],dfTravail[champCible])]
<ipython-input-77-406e00043759>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:


2021-07-19 16:21:47.975850 - temps de generation des possibles : 1.2903356552124023


In [148]:
dfCompletRec

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Cd_Cat,Catégorie_de_dépenses,Cd_Fam,...,12345689 OK,12345789 OK,12346789 OK,12356789 OK,12456789 OK,13456789 OK,23456789,23456789 OK,123456789,123456789 OK
475720,HU003,magyar cetelem bank,HU,CETMAGYAR.82110000,postai pénzforg. díj_paie,paie0,z-Moyens de paiement - autres,PAIE,Moyens de paiement,BANC,...,1,1,1,1,1,1,1,1,1,1
896038,FR8051,bddf - region grand est,-,-,checks subcontratcting fees,tchq,Traitement de chèques,TRVAL,Traitement des valeurs,BANC,...,1,1,1,1,1,1,1,1,1,1
1109843,US001,bank of the west,-,-,maint - atms only,gautm,Gestion d'automates,ELV,Prestations assurées par ELV,BANC,...,1,1,1,1,1,1,1,1,1,1
743195,RU41332,arval ooo,RU,ARVAL_RU.65232001,employee car - fuel,carbu,Carburant,AUTO,Voitures,SPRO,...,0,0,0,0,0,0,0,0,0,0
975925,LU45555,banque generale du luxembourg,-,-,issuing token debet card,carte,Cartes bancaires,PAIE,Moyens de paiement,BANC,...,1,1,1,1,1,1,1,1,1,1
870912,FR08081,bddf - pilotage especes & automates,-,-,funds transportation - atm maintenance,elv0,z-Prestations assurées par ELV - autres,ELV,Prestations assurées par ELV,BANC,...,1,1,1,1,1,1,1,1,1,1
1083171,BE13394,bnpp fortis,-,-,issuing linked services credit card,paie0,z-Moyens de paiement - autres,PAIE,Moyens de paiement,BANC,...,1,1,1,1,1,1,1,1,1,1
1132285,PT005,bnpp personal finance portugal,-,-,subcontrat informat - outros,srvval,"Services, externalisation, maintenance",TRVAL,Traitement des valeurs,BANC,...,0,0,0,0,0,0,1,0,1,0
603572,TH001,bnpp bangkok branch,TH,SAP.662360,swift,swift,Swift,U-TEL,Utilisation Télécoms,TECH,...,1,1,1,1,1,1,1,1,1,1
771540,BF22681,bici burkina faso,-,-,accueil personalise arrive depar,???,Non affecté à une famille précise,???,ZZ-Non affecté à une famille précise,???,...,1,1,1,1,1,1,1,1,1,1


In [149]:
dfResultatsTRec

,nbL,1,2,3,4,5,6,7,8,9,...,12345678,12345679,12345689,12345789,12346789,12356789,12456789,13456789,23456789,123456789
nbr,49,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,...,41.000000,41.000000,42.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
%,0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,83.673469,83.673469,85.714286,83.673469,83.673469,83.673469,83.673469,83.673469,83.673469,83.673469
nbr OK,0,41.000000,42.000000,42.000000,43.000000,41.000000,40.000000,44.000000,42.000000,42.000000,...,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000
% OK,0,83.673469,85.714286,85.714286,87.755102,83.673469,81.632653,89.795918,85.714286,85.714286,...,92.682927,92.682927,90.476190,92.682927,92.682927,92.682927,92.682927,92.682927,92.682927,92.682927


In [150]:
dfRCompilesRec = dfResultatsTRec.transpose().sort_values(by=['% OK', '%'], ascending=False)
dfRCompilesRec['% OK total'] = [a*b/100 for a,b in zip(dfRCompilesRec['%'],dfRCompilesRec['% OK'])]

In [155]:
dfRCompilesRec.sort_values(by=['% OK'], ascending=False)[:50]

,nbr,%,nbr OK,% OK,% OK total
247,43.0,87.755102,40.0,93.023256,81.632653
4579,43.0,87.755102,40.0,93.023256,81.632653
347,43.0,87.755102,40.0,93.023256,81.632653
45789,43.0,87.755102,40.0,93.023256,81.632653
12347,43.0,87.755102,40.0,93.023256,81.632653
4789,43.0,87.755102,40.0,93.023256,81.632653
5789,43.0,87.755102,40.0,93.023256,81.632653
2347,43.0,87.755102,40.0,93.023256,81.632653
1347,43.0,87.755102,40.0,93.023256,81.632653
1247,43.0,87.755102,40.0,93.023256,81.632653


In [156]:
dfRCompilesRec.loc['123456789']

nbr           41.000000
%             83.673469
nbr OK        38.000000
% OK          92.682927
% OK total    77.551020
Name: 123456789, dtype: float64

In [134]:
df9nonID = dfCompletRec[dfCompletRec['123456789']==0]

In [135]:
with Timer() as timerTout:
    dfCompletRec9nonID, dfResultatsTRec9nonID = traiteDFRetourRec(df9nonID, 'Acible', '123')
print()
print(str(datetime.now())+' - temps de generation des possibles : '+str(timerTout.interval))

<ipython-input-77-406e00043759>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTravail[i]=dfTravail[label]
<ipython-input-77-406e00043759>:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTravail[str(i)+' OK'] = [1 if a==b else 0 for a,b in zip(dfTravail[i],dfTravail[champCible])]
<ipython-input-77-406e00043759>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:


2021-07-19 15:57:22.526075 - temps de generation des possibles : 1.1330358982086182


In [136]:
dfRCompilesRec9nonID = dfResultatsTRec9nonID.transpose().sort_values(by=['% OK', '%'], ascending=False)
dfRCompilesRec9nonID['% OK total'] = [a*b/100 for a,b in zip(dfRCompilesRec9nonID['%'],dfRCompilesRec9nonID['% OK'])]

In [137]:
dfRCompilesRec9nonID.sort_values(by=['% OK total','%'], ascending=False)[:20]

,nbr,%,nbr OK,% OK,% OK total
1,449.0,100.000000,229.0,51.002227,51.002227
2,449.0,100.000000,224.0,49.888641,49.888641
3,449.0,100.000000,224.0,49.888641,49.888641
23,449.0,100.000000,224.0,49.888641,49.888641
9,449.0,100.000000,210.0,46.770601,46.770601
6,449.0,100.000000,205.0,45.657016,45.657016
12,385.0,85.746102,202.0,52.467532,44.988864
13,385.0,85.746102,202.0,52.467532,44.988864
123,385.0,85.746102,202.0,52.467532,44.988864
4,449.0,100.000000,197.0,43.875278,43.875278


In [231]:
dfTravail

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Cd_Cat,Catégorie_de_dépenses,Cd_Fam,...,12356789,12456789,13456789,12345679 OK,12345689 OK,12345789 OK,12346789 OK,12356789 OK,12456789 OK,13456789 OK
1139449,PT008,uci efc lisboa branch,-,-,asesor­a externa jur­dica,juri0,z-Honoraires juridiques - autres,JURID,Honoraires juridiques,SPRO,...,1,1,1,1,1,1,1,1,1,1
1401712,PL45551,bnpp bank polska,-,-,travel expenses - external transportation - do...,voya0,z-Voyages - autres,VOYA0,Z-Voyages - autres,VOYAG,...,1,1,1,1,1,1,1,1,1,1
225964,SG004,bnpp singapore branch,SG,SAP.615300,subsistence - meals (restaurants),resto,Restaurants,RESTO,Restaurants,VOYAG,...,1,1,1,1,1,1,1,1,1,1
1306590,HK011,bnpp hong kong branch,-,-,travel - taxi (business) - oversea,taxi,"Taxi, transports en commun",TAXI,"Taxi, transports en commun",VOYAG,...,1,1,1,1,1,1,1,1,1,1
728776,BE13394,bnpp fortis,BE,FORTIS.51655411,sport sponsorship,spnsr,Sponsoring,SPNSR,"Mécénat, sponsoring",C&MK,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334422,PL45551,bnpp bank polska,-,-,travel expenses - external transportation - do...,voya0,z-Voyages - autres,VOYA0,Z-Voyages - autres,VOYAG,...,1,1,1,1,1,1,1,1,1,1
940001,IN003,bnp paribas india solutions private ltd,-,-,mobile telephone calls,mobil,Mobiles,U-TEL,Utilisation Télécoms,TECH,...,1,1,1,0,0,0,0,0,0,0
1174545,TR45572,bnpp finansal kiralama as,-,-,other disallowable expenses,mgen0,z-Prestations externalisées logistique - autres,MGENE,Prestations externalisées logistique,C&MG,...,1,1,1,1,1,1,1,1,1,1
478347,IT033,banca nazionale del lavoro,IT,BNL.7180000004,oneri portabilita' rinegoziazione mutui,spro0,z-Services professionnels - autres,SPRO0,Z-Services professionnels - autres,SPRO,...,1,1,1,1,1,1,1,1,1,1


In [123]:
dfComplet, dfResultatsT = traiteDFRetour(dfTravail, 'Family Stream')

In [237]:
dfComplet

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Cd_Cat,Catégorie_de_dépenses,Cd_Fam,...,12356789,12456789,13456789,12345679 OK,12345689 OK,12345789 OK,12346789 OK,12356789 OK,12456789 OK,13456789 OK
1139449,PT008,uci efc lisboa branch,-,-,asesor­a externa jur­dica,juri0,z-Honoraires juridiques - autres,JURID,Honoraires juridiques,SPRO,...,1,1,1,1,1,1,1,1,1,1
1401712,PL45551,bnpp bank polska,-,-,travel expenses - external transportation - do...,voya0,z-Voyages - autres,VOYA0,Z-Voyages - autres,VOYAG,...,1,1,1,1,1,1,1,1,1,1
225964,SG004,bnpp singapore branch,SG,SAP.615300,subsistence - meals (restaurants),resto,Restaurants,RESTO,Restaurants,VOYAG,...,1,1,1,1,1,1,1,1,1,1
1306590,HK011,bnpp hong kong branch,-,-,travel - taxi (business) - oversea,taxi,"Taxi, transports en commun",TAXI,"Taxi, transports en commun",VOYAG,...,1,1,1,1,1,1,1,1,1,1
728776,BE13394,bnpp fortis,BE,FORTIS.51655411,sport sponsorship,spnsr,Sponsoring,SPNSR,"Mécénat, sponsoring",C&MK,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1334422,PL45551,bnpp bank polska,-,-,travel expenses - external transportation - do...,voya0,z-Voyages - autres,VOYA0,Z-Voyages - autres,VOYAG,...,1,1,1,1,1,1,1,1,1,1
940001,IN003,bnp paribas india solutions private ltd,-,-,mobile telephone calls,mobil,Mobiles,U-TEL,Utilisation Télécoms,TECH,...,1,1,1,0,0,0,0,0,0,0
1174545,TR45572,bnpp finansal kiralama as,-,-,other disallowable expenses,mgen0,z-Prestations externalisées logistique - autres,MGENE,Prestations externalisées logistique,C&MG,...,1,1,1,1,1,1,1,1,1,1
478347,IT033,banca nazionale del lavoro,IT,BNL.7180000004,oneri portabilita' rinegoziazione mutui,spro0,z-Services professionnels - autres,SPRO0,Z-Services professionnels - autres,SPRO,...,1,1,1,1,1,1,1,1,1,1


In [126]:
dfResultatsT

,nbL,9_mm,1,2,3,4,5,6,7,8,...,3456789_mm,12345678_mm,12345679_mm,12345689_mm,12345789_mm,12346789_mm,12356789_mm,12456789_mm,13456789_mm,23456789_mm
nbr,13731,13432.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13433.000000,13435.000000,13460.000000,13453.000000,13458.000000,13453.000000,13464.000000,13432.000000,13432.000000,13433.000000
%,0,97.822446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,97.829728,97.844294,98.026364,97.975384,98.011798,97.975384,98.055495,97.822446,97.822446,97.829728
nbr OK,0,12978.000000,13140.000000,13142.000000,13142.000000,13126.000000,13129.00000,13148.000000,13130.000000,13114.000000,...,12979.000000,12980.000000,12997.000000,12988.000000,12990.000000,12990.000000,12993.000000,12978.000000,12978.000000,12979.000000
% OK,0,96.620012,95.695871,95.710436,95.710436,95.593912,95.61576,95.754133,95.623043,95.506518,...,96.620264,96.613323,96.560178,96.543522,96.522514,96.558388,96.501783,96.620012,96.620012,96.620264


In [127]:
dfRCompiles = dfResultatsT.transpose().sort_values(by=['% OK', '%'], ascending=False)
dfRCompiles['% OK total'] = [a*b/100 for a,b in zip(dfRCompiles['%'],dfRCompiles['% OK'])]

In [220]:
#dfRCompiles.sort_values(by=['% OK', '%', '% OK total'], ascending=False)
dfRCompiles.sort_values(by=['% OK'], ascending=False)

,nbr,%,nbr OK,% OK,% OK total
2456789_mm,13433.0,97.829728,12979.0,96.620264,94.523341
23456789_mm,13433.0,97.829728,12979.0,96.620264,94.523341
3456789_mm,13433.0,97.829728,12979.0,96.620264,94.523341
9_mm,13432.0,97.822446,12978.0,96.620012,94.516059
13456789_mm,13432.0,97.822446,12978.0,96.620012,94.516059
...,...,...,...,...,...
7,NaN,NaN,13130.0,95.623043,NaN
5,NaN,NaN,13129.0,95.615760,NaN
4,NaN,NaN,13126.0,95.593912,NaN
8,NaN,NaN,13114.0,95.506518,NaN


In [109]:
dfRetour[1]=dfRetour['LogisticRegression_liblinear_1']
dfRetour[2]=dfRetour['LogisticRegression_sag_1']
dfRetour[3]=dfRetour['LogisticRegression_saga_1']
dfRetour[4]=dfRetour['PassiveAggressive_1']
dfRetour[5]=dfRetour['SGD_hinge_1']
dfRetour[6]=dfRetour['SGD_log_1']
dfRetour[7]=dfRetour['SGD_modified_huber_1']
dfRetour[8]=dfRetour['LinearSVC_hinge_1']
dfRetour[9]=dfRetour['LinearSVC_squared_hinge_1']

dfTravail = dfRetour.copy()

In [167]:
def traiteDFRetour(dfTravail, champCible):
    nbLignes = dfTravail.shape[0]

    dicoTemp={}
    dicoTemp['nbL'] = {}
    dicoTemp['nbL']['nbr'] = nbLignes
    dicoTemp['nbL']['%'] = 0
    dicoTemp['nbL']['nbr OK'] = 0
    dicoTemp['nbL']['% OK'] = 0

    ## TOUS
    #si les 9 sont identiques
    dfTravail['9_mm']=[1 if a==b==c==d==e==f==g==h==i else 0 for a,b,c,d,e,f,g,h,i in zip(dfTravail[1],dfTravail[2],dfTravail[3],dfTravail[4],dfTravail[5],dfTravail[6],dfTravail[7],dfTravail[8],dfTravail[9])]

    dicoTemp['9_mm'] = {}
    dicoTemp['9_mm']['nbr'] = sum(dfTravail['9_mm'])
    dicoTemp['9_mm']['%'] = 100*sum(dfTravail['9_mm'])/nbLignes

    #si chacun est pareil que cible, résultat attendu
    for i in range(1,10):
        dfTravail[str(i)+'_OK'] = [1 if a==b else 0 for a,b in zip(dfTravail[i],dfTravail[champCible])]
        dicoTemp[str(i)] = {}
        dicoTemp[str(i)]['nbr OK'] = sum(dfTravail[str(i)+'_OK'])
        dicoTemp[str(i)]['% OK'] = 100*sum(dfTravail[str(i)+'_OK'])/nbLignes

    #si les 9 sont identiques et bons, sont donc tous bons
    dfTravail['9_mm et OK']=[a*b for a,b in zip(dfTravail['1_OK'],dfTravail['9_mm'])]
    dicoTemp['9_mm']['nbr OK'] = sum(dfTravail['9_mm et OK'])
    dicoTemp['9_mm']['% OK'] = 100*sum(dfTravail['9_mm et OK'])/sum(dfTravail['9_mm'])

    ## 2
    #si sont identiques par 2
    for i in range(1,10):
        for j in range(i+1,10):
            label = str(i)+str(j)
            dfTravail[label+'_mm']=[1 if a==b else 0 for a,b in zip(dfTravail[i],dfTravail[j])]
            dicoTemp[label+'_mm'] = {}
            dicoTemp[label+'_mm']['nbr'] = sum(dfTravail[label+'_mm'])
            dicoTemp[label+'_mm']['%'] = 100*sum(dfTravail[label+'_mm'])/nbLignes
    #si sont tous les 2 bons
    for i in range(1,10):
        for j in range(i+1,10):
            label = str(i)+str(j)
            dfTravail[label+'_mm et OK']=[a*b for a,b in zip(dfTravail[str(i)+'_OK'],dfTravail[label+'_mm'])]
            dicoTemp[label+'_mm']['nbr OK'] = sum(dfTravail[label+'_mm et OK'])
            dicoTemp[label+'_mm']['% OK'] = 100*sum(dfTravail[label+'_mm et OK'])/sum(dfTravail[label+'_mm'])

    ## 3
    #si sont identiques par 3
    for i in range(1,10):
        for j in range(i+1,10):
            for k in range(j+1,10):
                label = str(i)+str(j)+str(k)
                dfTravail[label+'_mm']=[1 if a==b==c else 0 for a,b, c in zip(dfTravail[i],dfTravail[j],dfTravail[k])]
                dicoTemp[label+'_mm'] = {}
                dicoTemp[label+'_mm']['nbr'] = sum(dfTravail[label+'_mm'])
                dicoTemp[label+'_mm']['%'] = 100*sum(dfTravail[label+'_mm'])/nbLignes
    #si sont tous les 3 bons
    for i in range(1,10):
        for j in range(i+1,10):
            for k in range(j+1,10):
                label = str(i)+str(j)+str(k)
                dfTravail[label+'_mm et OK']=[a*b for a,b in zip(dfTravail[str(i)+'_OK'],dfTravail[label+'_mm'])]
                dicoTemp[label+'_mm']['nbr OK'] = sum(dfTravail[label+'_mm et OK'])
                dicoTemp[label+'_mm']['% OK'] = 100*sum(dfTravail[label+'_mm et OK'])/sum(dfTravail[label+'_mm'])

    ## 4
    #si sont identiques par 4
    for i in range(1,10):
        for j in range(i+1,10):
            for k in range(j+1,10):
                for l in range(k+1,10):
                    label = str(i)+str(j)+str(k)+str(l)
                    dfTravail[label+'_mm']=[1 if a==b==c==d else 0 for a,b,c,d in zip(dfTravail[i],dfTravail[j],dfTravail[k],dfTravail[l])]
                    dicoTemp[label+'_mm'] = {}
                    dicoTemp[label+'_mm']['nbr'] = sum(dfTravail[label+'_mm'])
                    dicoTemp[label+'_mm']['%'] = 100*sum(dfTravail[label+'_mm'])/nbLignes
    #si sont tous les 4 bons
    for i in range(1,10):
        for j in range(i+1,10):
            for k in range(j+1,10):
                for l in range(k+1,10):
                    label = str(i)+str(j)+str(k)+str(l)
                    dfTravail[label+'_mm et OK']=[a*b for a,b in zip(dfTravail[str(i)+'_OK'],dfTravail[label+'_mm'])]
                    dicoTemp[label+'_mm']['nbr OK'] = sum(dfTravail[label+'_mm et OK'])
                    dicoTemp[label+'_mm']['% OK'] = 100*sum(dfTravail[label+'_mm et OK'])/sum(dfTravail[label+'_mm'])

    ## 5
    #si sont identiques par 5
    for i in range(1,10):
        for j in range(i+1,10):
            for k in range(j+1,10):
                for l in range(k+1,10):
                    for m in range(l+1,10):
                        label = str(i)+str(j)+str(k)+str(l)+str(m)
                        dfTravail[label+'_mm']=[1 if a==b==c==d==e else 0 for a,b,c,d,e in zip(dfTravail[i],dfTravail[j],dfTravail[k],dfTravail[l],dfTravail[m])]
                        dicoTemp[label+'_mm'] = {}
                        dicoTemp[label+'_mm']['nbr'] = sum(dfTravail[label+'_mm'])
                        dicoTemp[label+'_mm']['%'] = 100*sum(dfTravail[label+'_mm'])/nbLignes
    #si sont tous les 5 bons
    for i in range(1,10):
        for j in range(i+1,10):
            for k in range(j+1,10):
                for l in range(k+1,10):
                    for m in range(l+1,10):
                        label = str(i)+str(j)+str(k)+str(l)+str(m)
                        dfTravail[label+'_mm et OK']=[a*b for a,b in zip(dfTravail[str(i)+'_OK'],dfTravail[label+'_mm'])]
                        dicoTemp[label+'_mm']['nbr OK'] = sum(dfTravail[label+'_mm et OK'])
                        dicoTemp[label+'_mm']['% OK'] = 100*sum(dfTravail[label+'_mm et OK'])/sum(dfTravail[label+'_mm'])

    ## 6
    #si sont identiques par 6
    for i in range(1,10):
        for j in range(i+1,10):
            for k in range(j+1,10):
                for l in range(k+1,10):
                    for m in range(l+1,10):
                        for n in range(m+1,10):
                            label = str(i)+str(j)+str(k)+str(l)+str(m)+str(n)
                            dfTravail[label+'_mm']=[1 if a==b==c==d==e==f else 0 for a,b,c,d,e,f in zip(dfTravail[i],dfTravail[j],dfTravail[k],dfTravail[l],dfTravail[m],dfTravail[n])]
                            dicoTemp[label+'_mm'] = {}
                            dicoTemp[label+'_mm']['nbr'] = sum(dfTravail[label+'_mm'])
                            dicoTemp[label+'_mm']['%'] = 100*sum(dfTravail[label+'_mm'])/nbLignes
    #si sont tous les 6 bons
    for i in range(1,10):
        for j in range(i+1,10):
            for k in range(j+1,10):
                for l in range(k+1,10):
                    for m in range(l+1,10):
                        for n in range(m+1,10):
                            label = str(i)+str(j)+str(k)+str(l)+str(m)+str(n)
                            dfTravail[label+'_mm et OK']=[a*b for a,b in zip(dfTravail[str(i)+'_OK'],dfTravail[label+'_mm'])]
                            dicoTemp[label+'_mm']['nbr OK'] = sum(dfTravail[label+'_mm et OK'])
                            dicoTemp[label+'_mm']['% OK'] = 100*sum(dfTravail[label+'_mm et OK'])/sum(dfTravail[label+'_mm'])

    ## 7
    #si sont identiques par 7
    for i in range(1,10):
        for j in range(i+1,10):
            for k in range(j+1,10):
                for l in range(k+1,10):
                    for m in range(l+1,10):
                        for n in range(m+1,10):
                            for o in range(n+1,10):
                                label = str(i)+str(j)+str(k)+str(l)+str(m)+str(n)+str(o)
                                dfTravail[label+'_mm']=[1 if a==b==c==d==e==f==g else 0 for a,b,c,d,e,f,g in zip(dfTravail[i],dfTravail[j],dfTravail[k],dfTravail[l],dfTravail[m],dfTravail[n],dfTravail[o])]
                                dicoTemp[label+'_mm'] = {}
                                dicoTemp[label+'_mm']['nbr'] = sum(dfTravail[label+'_mm'])
                                dicoTemp[label+'_mm']['%'] = 100*sum(dfTravail[label+'_mm'])/nbLignes
    #si sont tous les 7 bons
    for i in range(1,10):
        for j in range(i+1,10):
            for k in range(j+1,10):
                for l in range(k+1,10):
                    for m in range(l+1,10):
                        for n in range(m+1,10):
                            for o in range(n+1,10):
                                label = str(i)+str(j)+str(k)+str(l)+str(m)+str(n)+str(o)
                                dfTravail[label+'_mm et OK']=[a*b for a,b in zip(dfTravail[str(i)+'_OK'],dfTravail[label+'_mm'])]
                                dicoTemp[label+'_mm']['nbr OK'] = sum(dfTravail[label+'_mm et OK'])
                                dicoTemp[label+'_mm']['% OK'] = 100*sum(dfTravail[label+'_mm et OK'])/sum(dfTravail[label+'_mm'])

    ## 8
    #si sont identiques par 8
    sommeMM = 0
    for i in range(1,10):
        for j in range(i+1,10):
            for k in range(j+1,10):
                for l in range(k+1,10):
                    for m in range(l+1,10):
                        for n in range(m+1,10):
                            for o in range(n+1,10):
                                for p in range(o+1,10):
                                    label = str(i)+str(j)+str(k)+str(l)+str(m)+str(n)+str(o)+str(p)
                                    dfTravail[label+'_mm']=[1 if a==b==c==d==e==f==g==h else 0 for a,b,c,d,e,f,g,h in zip(dfTravail[i],dfTravail[j],dfTravail[k],dfTravail[l],dfTravail[m],dfTravail[n],dfTravail[o],dfTravail[p])]
                                    dicoTemp[label+'_mm'] = {}
                                    dicoTemp[label+'_mm']['nbr'] = sum(dfTravail[label+'_mm'])
                                    dicoTemp[label+'_mm']['%'] = 100*sum(dfTravail[label+'_mm'])/nbLignes
    #si sont tous les 8 bons
    sommeOK = 0
    for i in range(1,10):
        for j in range(i+1,10):
            for k in range(j+1,10):
                for l in range(k+1,10):
                    for m in range(l+1,10):
                        for n in range(m+1,10):
                            for o in range(n+1,10):
                                for p in range(o+1,10):
                                    label = str(i)+str(j)+str(k)+str(l)+str(m)+str(n)+str(o)+str(p)
                                    dfTravail[label+'_mm et OK']=[a*b for a,b in zip(dfTravail[str(i)+'_OK'],dfTravail[label+'_mm'])]
                                    dicoTemp[label+'_mm']['nbr OK'] = sum(dfTravail[label+'_mm et OK'])
                                    dicoTemp[label+'_mm']['% OK'] = 100*sum(dfTravail[label+'_mm et OK'])/sum(dfTravail[label+'_mm'])

    dfResultatsT = pd.DataFrame.from_dict(dicoTemp)
    
    return dfTravail, dfResultatsT

In [163]:
print(df1821.groupby(['Family Stream']))
print(len(df1821.groupby(['Family Stream'])))
for i in df1821.groupby(['Family Stream']).count().transpose().columns:
    print(i)

9
banking services
bnp paribas group
corporate insurance
corporate services
market data
not procurable
others
professional services
technology


In [162]:
df1821.groupby(['Family Stream']).count()

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Cd_Cat,Catégorie_de_dépenses,Cd_Fam,...,Fournisseur,Fournisseur identification,SommeDekEUR,Category Stream,Sub Category Stream,AcléMin,AcléMinE,AcléEt,AcléEtSansE,Acible
Family Stream,,,,,,,,,,,,,,,,,,,,,
banking services,10184,10184,10184,10184,10184,10184,10184,10184,10184,10184,...,10184,10184,10184,10184,10184,10184,10184,10184,10184,10184
bnp paribas group,28646,28646,28646,28646,28646,28646,28646,28646,28646,28646,...,28646,28646,28646,28646,28646,28646,28646,28646,28646,28646
corporate insurance,18941,18941,18941,18941,18941,18941,18941,18941,18941,18941,...,18941,18941,18941,18941,18941,18941,18941,18941,18941,18941
corporate services,757196,757196,757196,757196,757196,757196,757196,757196,757196,757196,...,757196,757196,757196,757196,757196,757196,757196,757196,757196,757196
market data,21299,21299,21299,21299,21299,21299,21299,21299,21299,21299,...,21299,21299,21299,21299,21299,21299,21299,21299,21299,21299
not procurable,45167,45167,45167,45167,45167,45167,45167,45167,45167,45167,...,45167,45167,45167,45167,45167,45167,45167,45167,45167,45167
others,163,163,163,163,163,163,163,163,163,163,...,163,163,163,163,163,163,163,163,163,163
professional services,324518,324518,324518,324518,324518,324518,324518,324518,324518,324518,...,324518,324518,324518,324518,324518,324518,324518,324518,324518,324518
technology,166907,166907,166907,166907,166907,166907,166907,166907,166907,166907,...,166907,166907,166907,166907,166907,166907,166907,166907,166907,166907


In [31]:
dftoto['Status'] = ["coucou" if s == 'banking services' else "-" for s in dftoto['Family Stream']]

In [37]:
dftoto[dftoto['CodePays'] == 'PL']#[dftoto['Family Stream'] == 'banking services']

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Cd_Cat,Catégorie_de_dépenses,Cd_Fam,...,SommeDekEUR,Family Stream,Category Stream,Sub Category Stream,AcléMin,AcléMinE,AcléEt,AcléEtSansE,Acible,Status
0,PL45551,bnpp bank polska,PL,ROD.954M30,organizacja eventów i spotka?,rp,"Evénementiel, RP et services marketing",CSLTG,Prestations de conseil en communication,C&MK,...,0.14400,technology,hardware,-,bnpp bank polska ; organizacja eventów i spotk...,bnpp bank polska ; organizacja eventów i spotk...,bnpp bank polska ; organizacja eventów i spotk...,"organizacja eventów i spotka? ; "",,na brzegu""""...",technology ; hardware ; -,-
22,PL45551,bnpp bank polska,PL,ROD.954194,general-bank training - other,fpc0,z-Formation - autres,FPC,Formation,SPRO,...,0.76800,professional services,training,-,bnpp bank polska ; general-bank training - oth...,bnpp bank polska ; general-bank training - oth...,bnpp bank polska ; general-bank training - oth...,"general-bank training - other ; ""adam piskozub...",professional services ; training ; -,-
57,PL45551,bnpp bank polska,PL,ROD.954M40,promocja i reklama - atl - produkcja offline,rp,"Evénementiel, RP et services marketing",CSLTG,Prestations de conseil en communication,C&MK,...,0.04133,professional services,marcom,advertising (incl purchase of space & creation...,bnpp bank polska ; promocja i reklama - atl - ...,bnpp bank polska ; promocja i reklama - atl - ...,bnpp bank polska ; promocja i reklama - atl - ...,promocja i reklama - atl - produkcja offline ;...,professional services ; marcom ; advertising (...,-
108,PL45551,bnpp bank polska,PL,ROD.8212B1,?rodki trwa?e w budowie - projekty dzn,sftw0,z-Logiciels - autres,SOFTW,Logiciels,TECH,...,2.06388,technology,software,-,bnpp bank polska ; ?rodki trwa?e w budowie - ...,bnpp bank polska ; ?rodki trwa?e w budowie - ...,bnpp bank polska ; ?rodki trwa?e w budowie - ...,"?rodki trwa?e w budowie - projekty dzn ; ""''a...",technology ; software ; -,-
136,PL45551,bnpp bank polska,PL,ROD.954531,working meeting regarded as cost of income,conf,Voyages de groupe,CONF,Voyages de groupe,VOYAG,...,2.05961,corporate services,travel,travel others (not splitted to a specific cate...,bnpp bank polska ; working meeting regarded as...,bnpp bank polska ; working meeting regarded as...,bnpp bank polska ; working meeting regarded as...,"working meeting regarded as cost of income ; ""...",corporate services ; travel ; travel others (n...,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748476,PL45551,bnpp bank polska,PL,ROD.954501,travel expenses - external transportation - do...,voya0,z-Voyages - autres,VOYA0,Z-Voyages - autres,VOYAG,...,0.00149,corporate services,travel,travel others (not splitted to a specific cate...,bnpp bank polska ; travel expenses - external ...,bnpp bank polska ; travel expenses - external ...,bnpp bank polska ; travel expenses - external ...,travel expenses - external transportation - do...,corporate services ; travel ; travel others (n...,-
748478,PL45551,bnpp bank polska,PL,ROD.954501,travel expenses - external transportation - do...,voya0,z-Voyages - autres,VOYA0,Z-Voyages - autres,VOYAG,...,0.00149,corporate services,travel,travel others (not splitted to a specific cate...,bnpp bank polska ; travel expenses - external ...,bnpp bank polska ; travel expenses - external ...,bnpp bank polska ; travel expenses - external ...,travel expenses - external transportation - do...,corporate services ; travel ; travel others (n...,-
748479,PL45551,bnpp bank polska,PL,ROD.954501,travel expenses - external transportation - do...,voya0,z-Voyages - autres,VOYA0,Z-Voyages - autres,VOYAG,...,0.00140,corporate services,travel,travel others (not splitted to a specific cate...,bnpp bank polska ; travel expenses - external ...,bnpp bank polska ; travel expenses - external ...,bnpp bank polska ; travel expenses - external ...,travel expenses - external transportation - do...,corporate services ; trave

In [154]:
tableauLabels = ['LogisticRegression_liblinear_dgfdxvc','LogisticRegression_sag_1',\
                                     'LogisticRegression_saga_azer','PassiveAggressive_1','SGD_hinge_1',\
                                     'SGD_log_1','SGD_modified_huber_dsfg','LinearSVC_hinge_1',\
                                     'LinearSVC_squared_hinge_ùmlkjhgdfsqf']
[(i,label) for i,label in zip([x for x in range(1,len(tableauLabels)+1)],tableauLabels)]

[(1, 'LogisticRegression_liblinear_dgfdxvc'),
 (2, 'LogisticRegression_sag_1'),
 (3, 'LogisticRegression_saga_azer'),
 (4, 'PassiveAggressive_1'),
 (5, 'SGD_hinge_1'),
 (6, 'SGD_log_1'),
 (7, 'SGD_modified_huber_dsfg'),
 (8, 'LinearSVC_hinge_1'),
 (9, 'LinearSVC_squared_hinge_ùmlkjhgdfsqf')]

In [212]:
tabqq=[]
tabqq.append(1)
tabqq.append('aze')
tabqq.append(543)
tabqq.pop()
tabqq

[1, 'aze']

In [20]:
#initialisation des dictionnaires de modèles
dicModelLR={}
dicModelLR['EF']={}
dicModelLR['ET']={}
dicModelLR['MF']={}
dicModelLR['MT']={}

dicModelPAC={}
dicModelPAC['EF']={}
dicModelPAC['ET']={}
dicModelPAC['MF']={}
dicModelPAC['MT']={}

dicModelSGD={}
dicModelSGD['EF']={}
dicModelSGD['ET']={}
dicModelSGD['MF']={}
dicModelSGD['MT']={}

dicModelSVM={}
dicModelSVM['EF']={}
dicModelSVM['ET']={}
dicModelSVM['MF']={}
dicModelSVM['MT']={}

dicModelNN={}

dicModel={}
dicModel['LogisticRegression']={}
dicModel['LogisticRegression']['EF']={}
dicModel['LogisticRegression']['ET']={}
dicModel['PassiveAggressiveClassifier']={}
dicModel['PassiveAggressiveClassifier']['EF']={}
dicModel['PassiveAggressiveClassifier']['ET']={}
dicModel['SGDClassifier']={}
dicModel['SGDClassifier']['EF']={}
dicModel['SGDClassifier']['ET']={}
dicModel['LinearSVC']={}
dicModel['LinearSVC']['EF']={}
dicModel['LinearSVC']['ET']={}

In [ ]:
#TEST sur les others 2020

dfO = dfL.copy()

indexNames = dfO[(dfO['Family Stream'] != 'Others') | ((dfO['Family Stream'] == 'Others') & (dfO['Category Stream'] == 'Transaction Fees'))].index
dfO.drop(indexNames , inplace=True)

dfO.fillna("-", inplace=True)

In [ ]:
dfO

In [ ]:
dfO["AcléMinEC"]=dfO["Entité"]+" ; "+dfO["Compte"]
to_predict = vect.transform(dfO["AcléMinEC"])
dfO["predictOthers"] = modelLR.predict(to_predict)

In [ ]:
to_predictProba = modelLR.predict_proba(to_predict)
dfO["predictOthersProba"] = [max(p) for p in to_predictProba]

In [ ]:
dfO.to_excel('Predicted Others with Proba.xlsx')

In [19]:
#Vectorize (TF-IDF, ngrams 1-3, no stop words) using sklearn -->
#fréquence minimum d'apparition : min_df
vectorizer_EtFS = TfidfVectorizer(analyzer='word', ngram_range=(1,3),
                     min_df = 3, stop_words = 'english', sublinear_tf=True)

In [143]:
dump(df1821, 'df1821.dump')

['df1821.dump']

In [ ]:
premier=chaine(1,0.9202)
courant = premier
courant.setSuivant(chaine(3,0.9206))
courant=courant.suivant
courant.setSuivant(chaine(10,0.9214))
courant=courant.suivant
courant.setSuivant(chaine(30,0.9314))
courant=courant.suivant
courant.setSuivant(chaine(100,0.9215))
courant=courant.suivant
last=courant

maxM, place = getMaillonScoreMax(premier) 
print(maxM.valeur)
print(maxM.suivant.valeur)
(nVP, nVS) = newValAutourduMax(maxM)
print(nVP, nVS)

maxM.placeAvant(chaine(nVP,0.9266))
maxM.placeApres(chaine(nVS,0.9276))

courant=last
while True:
    print(courant.valeur)

    if courant.precedent != None:
        courant=courant.precedent
    else:
        break

#X_Tr, X_Te, y_Tr, y_Te = train_test_split(dfW["AcléEt"][0:10000], dfW["Family Stream"][0:10000], test_size = 0.02)

#with Timer() as timer:
#    vect_V = vectorizer_EtFS.fit(X_Tr)
#print(str(datetime.now())+' - temps de traitement du fitting : '+str(timer.interval))
#X_TrV = vect_V.transform(X_Tr)
#X_TeV = vect_V.transform(X_Te)

#apprendData(1, True, 'LogisticRegression', 'EF', X_TrV, X_TeV, y_Tr, y_Te)

In [ ]:
X_train_vectorized = X_train_vectorized_EtFS
X_test_vectorized = X_test_vectorized_EtFS
y_train = y_train_EtFS
y_test = y_test_EtFS

units = 10
this_activation = 'relu'

for this_alpha in [0.000001, 0.00001, 0.0001, 0.001, 0.01]:
    label = str(this_alpha)
    print(datetime.now())
    modelNN = MLPClassifier(solver='adam', activation = this_activation, alpha = this_alpha, hidden_layer_sizes = [units, units, units], learning_rate='adaptive')
    dicModelNN[label]=modelNN.fit(X_train_vectorized, y_train)

    print(datetime.now())
    print("    score train : "+str(dicModelNN[label].score(X_train_vectorized,y_train)))
    print("    score test : "+str(dicModelNN[label].score(X_test_vectorized,y_test)))

    dump(dicModelNN[label], 'modeleNN_3_10_'+label+'.modele')

# Logistic Regression

In [ ]:
labels = modelLR.classes_

In [ ]:
predictions = modelLR.predict(X_test_vectorized)
predictionsProba = modelLR.predict_proba(X_test_vectorized)

In [ ]:
list2nd=[{i:j for i,j in zip(x,labels)}.get(sorted(x,reverse=True)[1]) for x in predictionsProba]
proba2nd=[sorted(x,reverse=True)[1] for x in predictionsProba]
list3e=[{i:j for i,j in zip(x,labels)}.get(sorted(x,reverse=True)[2]) for x in predictionsProba]
proba3e=[sorted(x,reverse=True)[2] for x in predictionsProba]
list4e=[{i:j for i,j in zip(x,labels)}.get(sorted(x,reverse=True)[3]) for x in predictionsProba]
proba4e=[sorted(x,reverse=True)[3] for x in predictionsProba]
list5e=[{i:j for i,j in zip(x,labels)}.get(sorted(x,reverse=True)[4]) for x in predictionsProba]
proba5e=[sorted(x,reverse=True)[4] for x in predictionsProba]

In [ ]:
maxP = [max(p) for p in predictionsProba]
maxP

In [ ]:
predictionsTRAIN = modelLR.predict(X_train_vectorized)

toCompareTRAIN=pd.DataFrame([predictionsTRAIN,y_train])
toCompareTRAIN = toCompareTRAIN.transpose()
toCompareTRAIN["score"]=[1 if i == j else 0 for i,j in zip (toCompareTRAIN[0], toCompareTRAIN[1])]

100*toCompareTRAIN["score"].sum()/toCompareTRAIN["score"].shape

In [ ]:
modelLR.score(X_test_vectorized,y_test)

In [ ]:
toCompare=pd.DataFrame([predictions,y_test])
toCompare = toCompare.transpose()
toCompare["score"]=[1 if i == j else 0 for i,j in zip (toCompare[0], toCompare[1])]

100*toCompare["score"].sum()/toCompare["score"].shape

In [ ]:
#2nd
toCompare2=pd.DataFrame([list2nd,y_test])
toCompare2 = toCompare2.transpose()
toCompare2["score"]=[1 if i == j else 0 for i,j in zip (toCompare2[0], toCompare2[1])]

100*toCompare2["score"].sum()/toCompare2["score"].shape

In [ ]:
#3e
toCompare3=pd.DataFrame([list3e,y_test])
toCompare3 = toCompare3.transpose()
toCompare3["score"]=[1 if i == j else 0 for i,j in zip (toCompare3[0], toCompare3[1])]

100*toCompare3["score"].sum()/toCompare3["score"].shape

In [ ]:
#4e
toCompare4=pd.DataFrame([list4e,y_test])
toCompare4 = toCompare4.transpose()
toCompare4["score"]=[1 if i == j else 0 for i,j in zip (toCompare4[0], toCompare4[1])]

100*toCompare4["score"].sum()/toCompare4["score"].shape

In [ ]:
#5e
toCompare5=pd.DataFrame([list5e,y_test])
toCompare5 = toCompare5.transpose()
toCompare5["score"]=[1 if i == j else 0 for i,j in zip (toCompare5[0], toCompare5[1])]

100*toCompare5["score"].sum()/toCompare5["score"].shape

In [ ]:
toCompare

In [ ]:
indexNames = toCompare[(toCompare['score'] == 0)].index
toCompare1 = toCompare.drop(indexNames)
indexNames = toCompare[(toCompare['score'] == 1)].index
toCompare0 = toCompare.drop(indexNames)

In [ ]:
print("count P(0): "+str(len(toCompare0["proba"])))
print("min P(0): "+str(min(toCompare0["proba"])))
print("max P(0): "+str(max(toCompare0["proba"])))
print("mean P(0): "+str(statistics.mean(toCompare0["proba"])))
print("median P(0): "+str(statistics.median(toCompare0["proba"])))
print()
print("count P(1): "+str(len(toCompare1["proba"])))
print("min P(1): "+str(min(toCompare1["proba"])))
print("max P(1): "+str(max(toCompare1["proba"])))
print("mean P(1): "+str(statistics.mean(toCompare1["proba"])))
print("median P(1): "+str(statistics.median(toCompare1["proba"])))

In [ ]:
toto=multilabel_confusion_matrix(y_testMin, predictionsMin)

# APPRENTISSAGE POUR RUN

In [121]:
def apprendModeles(X, y, labelDataApprises):
    vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3),
                         min_df = 3, stop_words = 'english', sublinear_tf=True)

    print(str(datetime.now())+" - __start__")
    with Timer() as timer:
        vectRun = vectorizer.fit(X)
    print(str(datetime.now())+' - temps de traitement du fitting : '+str(timer.interval))
    with Timer() as timer:
        X_RUN = vectRun.transform(X)
    print(str(datetime.now())+' - temps de traitement du transform : '+str(timer.interval))

    modelLRRun = LogisticRegression(solver='saga', C=35, max_iter=800)
    modelPACRun = PassiveAggressiveClassifier(C=0.65, max_iter=200)
    modelSGDCRun = SGDClassifier(alpha=0.0000001, max_iter=200)
    modelLSVCRun = LinearSVC(max_iter = 1000, C=6)

    with Timer() as timer:
        modelLRRun.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelLRRun : '+str(timer.interval))
    dump(modelLRRun, 'modeleLR_'+labelDataApprises+'.modele')

    with Timer() as timer:
        modelPACRun.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelPACRun : '+str(timer.interval))
    dump(modelPACRun, 'modelePAC_'+labelDataApprises+'.modele')

    with Timer() as timer:
        modelSGDCRun.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelSGDCRun : '+str(timer.interval))
    dump(modelSGDCRun, 'modeleSGD_'+labelDataApprises+'.modele')

    with Timer() as timer:
        modelLSVCRun.fit(X_RUN, y)
    print(str(datetime.now())+' - temps de traitement du fit de modelLSVCRun : '+str(timer.interval))
    dump(modelLSVCRun, 'modeleLSVC_'+labelDataApprises+'.modele')
    
    return vectRun, modelLRRun, modelPACRun, modelSGDCRun, modelLSVCRun

def preditStream(vecteur, X, modelLR, modelPAC, modelSGD, modelLSVC):
    with Timer() as timer:
        X_to_predict = vecteur.transform(X)
    print(str(datetime.now())+' - temps de traitement du transform : '+str(timer.interval))
    
    with Timer() as timer:
        predictedLR = modelLR.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelLRRun : '+str(timer.interval))
    with Timer() as timer:
        predictedPAC = modelPAC.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelPACRun : '+str(timer.interval))
    with Timer() as timer:
        predictedSGD = modelSGD.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelSGDCRun : '+str(timer.interval))
    with Timer() as timer:
        predictedSVC = modelLSVC.predict(X_to_predict)
    print(str(datetime.now())+' - temps de prediction de modelLSVCRun : '+str(timer.interval))
    
    return predictedLR, predictedPAC, predictedSGD, predictedSVC

In [108]:
vecteur1821, modelLR1821, modelPAC1821, modelSGDC1821, modelLSVC1821 = apprendModeles(df1821["AcléEt"], df1821["Acible"], '2018-2021Q1 sans others')

2021-05-19 17:14:43.663246 - __start__
2021-05-19 17:15:55.082340 - temps de traitement du fitting : 71.41909384727478
2021-05-19 17:17:00.512782 - temps de traitement du transform : 65.43044209480286
2021-05-19 18:27:51.948015 - temps de traitement du fit de modelLRRun : 4251.433210372925
2021-05-19 18:35:04.599009 - temps de traitement du fit de modelPACRun : 431.80424451828003
2021-05-19 18:43:02.259044 - temps de traitement du fit de modelSGDCRun : 476.9823799133301
2021-05-19 19:22:49.826386 - temps de traitement du fit de modelLSVCRun : 2386.5152292251587


In [114]:
df1821["LR_predict"], df1821["PAC_predict"], df1821["SGD_predict"], df1821["SVC_predict"] = preditStream(vecteur1821, df1821["AcléEt"], modelLR1821, modelPAC1821, modelSGDC1821, modelLSVC1821)

2021-05-19 19:44:38.384437 - temps de traitement du transform : 69.28543639183044
2021-05-19 19:44:44.748890 - temps de prediction de modelLRRun : 6.363452672958374
2021-05-19 19:44:50.761040 - temps de prediction de modelPACRun : 6.011150598526001
2021-05-19 19:44:56.786692 - temps de prediction de modelSGDCRun : 6.025651931762695
2021-05-19 19:45:00.974301 - temps de prediction de modelLSVCRun : 4.186635732650757


In [115]:
df1821

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Cd_Cat,Catégorie_de_dépenses,Cd_Fam,...,SommeDekEUR,Family Stream,Category Stream,Sub Category Stream,AcléEt,Acible,LR_predict,PAC_predict,SGD_predict,SVC_predict
0,PL45551,bnpp bank polska,PL,ROD.954M30,organizacja eventów i spotka?,rp,"Evénementiel, RP et services marketing",CSLTG,Prestations de conseil en communication,C&MK,...,0.14400,Technology,Hardware,-,bnpp bank polska ; organizacja eventów i spotk...,Technology ; Hardware ; -,Technology ; Hardware ; -,Technology ; Hardware ; -,Technology ; Hardware ; -,Technology ; Hardware ; -
1,ES018,uci espana,ES,UCI.62110,alquiler locales,loyer,Loyer,LOYER,Coûts d'occupation,IMMO,...,82.32000,Corporate Services,Corporate Real Estate,Costs of occupancy,"uci espana ; alquiler locales ; ""3491 ma, s.l""...",Corporate Services ; Corporate Real Estate ; C...,Corporate Services ; Corporate Real Estate ; C...,Corporate Services ; Corporate Real Estate ; C...,Corporate Services ; Corporate Real Estate ; C...,Corporate Services ; Corporate Real Estate ; C...
2,FR084,bnpp personal finance,FR,OCAPP.634622,presta informatiques refacturees ips specifiqu...,amoe,Assistance à la maîtrise d'oeuvre,CONSU,"Conseil, assistance technique",SPRO,...,131501.51836,BNP PARIBAS GROUP,-,-,bnpp personal finance ; presta informatiques r...,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -
3,FR060,bnpp imex,FR,SAP_IMEX.I621100RE0,loyers locaux d'exp,loyer,Loyer,LOYER,Coûts d'occupation,IMMO,...,114859.49123,BNP PARIBAS GROUP,-,-,bnpp imex ; loyers locaux d'exp ; bnp paribas ...,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -
4,FR084,bnpp personal finance,FR,OCAPP.634622,presta informatiques refacturees ips specifiqu...,amoe,Assistance à la maîtrise d'oeuvre,CONSU,"Conseil, assistance technique",SPRO,...,108750.00256,BNP PARIBAS GROUP,-,-,bnpp personal finance ; presta informatiques r...,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469482,IN43974,arval india private ltd,-,-,staff - company celebration,asoc0,z-Avantages sociaux - autres,A-SOC,Avantages sociaux,SPRO,...,0.16687,Corporate Insurance,-,-,arval india private ltd ; staff - company cele...,Corporate Insurance ; - ; -,Corporate Insurance ; - ; -,Corporate Services ; General Services ; Postage,Corporate Services ; General Services ; Postage,Corporate Insurance ; - ; -
1469483,IN43974,arval india private ltd,-,-,prof. fees - legal fees,juri0,z-Honoraires juridiques - autres,JURID,Honoraires juridiques,SPRO,...,0.61958,Corporate Insurance,-,-,arval india private ltd ; prof. fees - legal f...,Corporate Insurance ; - ; -,Corporate Insurance ; - ; -,Professional Services ; Legal ; -,Professional Services ; Legal ; -,Corporate Insurance ; - ; -
1469513,RU43677,cardif insurance company llc,-,-,opex marketing consulting and channels testing,c&mk0,z-Communication et Marketing - autres,C&MK0,Z-Communication et Marketing - autres,C&MK,...,31.91862,Professional Services,Non IT Consultancy (excl. EMC),-,cardif insurance company llc ; opex marketing ...,Professional Services ; Non IT Consultancy (ex...,Professional Services ; MarCom ; Advertising (...,Professional Services ; MarCom ; Advertising (...,Professional Services ; MarCom ; Advertising (...,Professional Services ; MarCom ; Advertising (...
1469514,RU43677,cardif insurance company llc,-,-,opex other consulting,cons0,"z-Conseil, assistance technique - autres",CONSU,"Conseil, assistance technique",SPRO,...,113.51913,Professional Services,Non IT Consultancy (excl. EMC),-,cardif insurance company llc ; opex other cons...,Professional Services ; Non IT Consultancy (ex...,Technology ; IT Services ; -,Technology 

In [112]:
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3),
                     min_df = 3, stop_words = 'english', sublinear_tf=True)

#apprend sur toute la base
vectRun = vectorizer.fit(df1821["AcléEt"])

In [110]:
to_predictRun1821 = vectRun.transform(df1821["AcléEt"])

with Timer() as timer:
    df1821["LR_predict"] = modelLRRun.predict(to_predictRun1821)
print(str(datetime.now())+' - temps de prediction de modelLRRun : '+str(timer.interval))
with Timer() as timer:
    df1821["PAC_predict"] = modelPACRun.predict(to_predictRun1821)
print(str(datetime.now())+' - temps de prediction de modelPACRun : '+str(timer.interval))
with Timer() as timer:
    df1821["SGD_predict"] = modelSGDCRun.predict(to_predictRun1821)
print(str(datetime.now())+' - temps de prediction de modelSGDCRun : '+str(timer.interval))
with Timer() as timer:
    df1821["SVC_predict"] = modelLSVCRun.predict(to_predictRun1821)
print(str(datetime.now())+' - temps de prediction de modelLSVCRun : '+str(timer.interval))

ValueError: X has 579238 features per sample; expecting 1174421

In [ ]:
vecteur1921, modelLR1921, modelPAC1921, modelSGDC1921, modelLSVC1921 = apprendModeles(df1921["AcléEt"], df1921["Acible"], '2019-2021Q1 sans others')

In [116]:
df1617courtBase = pd.read_excel('./Lignes streamées manuellement sur 2016 & 2017.xlsx')

In [153]:
df1617court = df1617courtBase

df1617court["Entité"] = df1617court["Entité"].str.lower()
df1617court["Compte"] = df1617court["Compte"].str.lower()
df1617court["Fournisseur identification"] = df1617court["Fournisseur identification"].str.lower()
df1617court["Cd_Scat"] = df1617court["Cd_Scat"].str.lower()
df1617court["Fournisseur"] = df1617court["Fournisseur"].str.lower()

#remplis les vide avec un tiret "-"
df1617court.fillna("-", inplace=True)

df1617court["AcléEt"]=df1617court["Entité"]+" ; "+df1617court["Compte"]+" ; "+df1617court["Fournisseur identification"]+" ; "+df1617court["Cd_Scat"]+" ; "+df1617court["Fournisseur"]
df1617court

,CdEnt,Entité,CLE_COMPTE,Compte,Cd_Scat,Fournisseur,Fournisseur identification,kEUR,Family Stream,Category Stream,Sub Category Stream,AcléEt
0,FR023,cortal consors france,SAP_IMEX.607115,prestations de service informatiques au forfait,amoe,4tpm,4tpm,133.23320,Technology,IT Services,-,cortal consors france ; prestations de service...
1,FR307,bddf - operations,SAP_IMEX.607115,prestations de service informatiques au forfait,amoe,4tpm,4tpm,27.30770,Technology,IT Services,-,bddf - operations ; prestations de service inf...
2,BE13394,bnpp fortis,FORTIS.17362100,software 5 years,sftw0,ab initio,ab initio software,598.71700,Technology,Software,-,bnpp fortis ; software 5 years ; ab initio sof...
3,BE13394,bnpp fortis,FORTIS.17362100,software 5 years,sftw0,ab initio software sarl,ab initio software sarl,58.33575,Technology,Software,-,bnpp fortis ; software 5 years ; ab initio sof...
4,IT033,banca nazionale del lavoro,BNL.7250000148,assistenza informatica non a progetto,mtsrv,avanade italy s.r.l.,accenture,152.80000,Technology,IT Services,-,banca nazionale del lavoro ; assistenza inform...
...,...,...,...,...,...,...,...,...,...,...,...,...
588,BE13394,bnpp fortis,FORTIS.51695211,issuing transactions costs debet card,paie0,atos worldline sa/nv,worldline,422.89724,Banking Services,Credit Cards (incl Processing),-,bnpp fortis ; issuing transactions costs debet...
589,FR8444,bddf - banque en ligne,SAP_IMEX.658200,informatique achats licences,lcnce,wt emea acquisition ltd,wt emea acquisition limited,148.00000,Technology,Software,-,bddf - banque en ligne ; informatique achats l...
590,FR308,bddf - retail,OCAPP.658200,softwares - purchase,lcnce,wt emea acquisition ltd,wt emea acquisition limited,150.00000,Technology,Software,-,bddf - retail ; softwares - purchase ; wt emea...
591,FR308,bddf - retail,OCAPP.607150,tailoring work it,amoe,wt emea acquisition ltd,wt emea acquisition limited,150.00000,Technology,IT Services,-,bddf - retail ; tailoring work it ; wt emea ac...


In [156]:
df1617court["LR_predict"], df1617court["PAC_predict"], df1617court["SGD_predict"], df1617court["SVC_predict"] = preditStream(vecteur1821, df1617court["AcléEt"], modelLR1821, modelPAC1821, modelSGDC1821, modelLSVC1821)

2021-05-19 20:13:45.006295 - temps de traitement du transform : 0.08451008796691895
2021-05-19 20:13:45.848818 - temps de prediction de modelLRRun : 0.8415226936340332
2021-05-19 20:13:46.594592 - temps de prediction de modelPACRun : 0.7447597980499268
2021-05-19 20:13:47.272790 - temps de prediction de modelSGDCRun : 0.6781983375549316
2021-05-19 20:13:47.275760 - temps de prediction de modelLSVCRun : 0.0019474029541015625


In [157]:
df1617court

,CdEnt,Entité,CLE_COMPTE,Compte,Cd_Scat,Fournisseur,Fournisseur identification,kEUR,Family Stream,Category Stream,Sub Category Stream,AcléEt,LR_predict,PAC_predict,SGD_predict,SVC_predict
0,FR023,cortal consors france,SAP_IMEX.607115,prestations de service informatiques au forfait,amoe,4tpm,4tpm,133.23320,Technology,IT Services,-,cortal consors france ; prestations de service...,Technology ; IT Services ; -,Technology ; IT Services ; -,Technology ; IT Services ; -,Technology ; IT Services ; -
1,FR307,bddf - operations,SAP_IMEX.607115,prestations de service informatiques au forfait,amoe,4tpm,4tpm,27.30770,Technology,IT Services,-,bddf - operations ; prestations de service inf...,Technology ; IT Services ; -,Technology ; IT Services ; -,Technology ; IT Services ; -,Technology ; IT Services ; -
2,BE13394,bnpp fortis,FORTIS.17362100,software 5 years,sftw0,ab initio,ab initio software,598.71700,Technology,Software,-,bnpp fortis ; software 5 years ; ab initio sof...,Technology ; Software ; -,Technology ; Software ; -,Technology ; Software ; -,Technology ; Software ; -
3,BE13394,bnpp fortis,FORTIS.17362100,software 5 years,sftw0,ab initio software sarl,ab initio software sarl,58.33575,Technology,Software,-,bnpp fortis ; software 5 years ; ab initio sof...,Technology ; Software ; -,Technology ; Software ; -,Technology ; Software ; -,Technology ; Software ; -
4,IT033,banca nazionale del lavoro,BNL.7250000148,assistenza informatica non a progetto,mtsrv,avanade italy s.r.l.,accenture,152.80000,Technology,IT Services,-,banca nazionale del lavoro ; assistenza inform...,Technology ; IT Services ; -,Technology ; IT Services ; -,Technology ; IT Services ; -,Technology ; IT Services ; -
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,BE13394,bnpp fortis,FORTIS.51695211,issuing transactions costs debet card,paie0,atos worldline sa/nv,worldline,422.89724,Banking Services,Credit Cards (incl Processing),-,bnpp fortis ; issuing transactions costs debet...,Banking Services ; Banking software & services...,Banking Services ; Banking software & services...,Banking Services ; Banking software & services...,Banking Services ; Banking software & services...
589,FR8444,bddf - banque en ligne,SAP_IMEX.658200,informatique achats licences,lcnce,wt emea acquisition ltd,wt emea acquisition limited,148.00000,Technology,Software,-,bddf - banque en ligne ; informatique achats l...,Technology ; Software ; -,Technology ; Software ; -,Technology ; Software ; -,Technology ; Software ; -
590,FR308,bddf - retail,OCAPP.658200,softwares - purchase,lcnce,wt emea acquisition ltd,wt emea acquisition limited,150.00000,Technology,Software,-,bddf - retail ; softwares - purchase ; wt emea...,Technology ; Software ; -,Technology ; Software ; -,Technology ; Software ; -,Technology ; Software ; -
591,FR308,bddf - retail,OCAPP.607150,tailoring work it,amoe,wt emea acquisition ltd,wt emea acquisition limited,150.00000,Technology,IT Services,-,bddf - retail ; tailoring work it ; wt emea ac...,Technology ; IT Services ; -,Technology ; IT Services ; -,Technology ; IT Services ; -,Technology ; IT Services ; -


In [158]:
df1617court.to_excel('2016-2017 verif des streams.xlsx')

In [160]:
df1617Base = pd.read_excel('./Lignes 2016 & 2017 à streamer avec l algo ML.xlsx')

In [161]:
df1617 = df1617Base

df1617["Entité"] = df1617["Entité"].str.lower()
df1617["Compte"] = df1617["Compte"].str.lower()
df1617["Fournisseur identification"] = df1617["Fournisseur identification"].str.lower()
df1617["Cd_Scat"] = df1617["Cd_Scat"].str.lower()
df1617["Fournisseur"] = df1617["Fournisseur"].str.lower()

#remplis les vide avec un tiret "-"
df1617.fillna("-", inplace=True)

df1617["AcléEt"]=df1617["Entité"]+" ; "+df1617["Compte"]+" ; "+df1617["Fournisseur identification"]+" ; "+df1617["Cd_Scat"]+" ; "+df1617["Fournisseur"]
df1617

,CdEnt,Entité,CodePays,CdCompte,CLE_COMPTE,Compte,Cd_Scat,Expense_Sub_Category,Cd_Cat,Expense_Category,...,Country,Region,Key Cpte + Four,Année,Amount Category,Family Stream,Category Stream,Sub Category Stream,ABS Amount,AcléEt
0,FR8444,bddf - banque en ligne,FR,SAP_IMEX.607140.FR8444,SAP_IMEX.607140,sous traitance - services divers,spro0,z-Professional services - others,SPRO0,Z-Professional services - others,...,France,France,SAP_IMEX.607140SAP_IMEX.100059654,2016,<= 100K€,-,-,-,99.80000,bddf - banque en ligne ; sous traitance - serv...
1,FR101,bddf - fonctions,FR,SAP_IMEX.607112.FR101,SAP_IMEX.607112,prestation d'assistance extérieure informatique,amoe,Assistance à la maîtrise d'oeuvre,CONSU,Consultants,...,France,France,SAP_IMEX.607112SAP_IMEX.100038426,2016,<= 100K€,-,-,-,99.75000,bddf - fonctions ; prestation d'assistance ext...
2,FR317,bddf - dr lyon,FR,SAP_IMEX.615440.FR317,SAP_IMEX.615440,réception - autres resto. - employés bnpp,resto,Restaurants,RESTO,Restaurants,...,France,France,SAP_IMEX.615440SAP_IMEX.100025945,2016,<= 100K€,-,-,-,99.25546,bddf - dr lyon ; réception - autres resto. - e...
3,FR513,bddf - entreprises,FR,SAP_IMEX.607140.FR513,SAP_IMEX.607140,sous traitance - services divers,spro0,z-Professional services - others,SPRO0,Z-Professional services - others,...,France,France,SAP_IMEX.607140SAP_IMEX.100082376,2017,<= 100K€,-,-,-,98.48634,bddf - entreprises ; sous traitance - services...
4,BE13394,bnpp fortis,BE,FORTIS.51670127.BE13394,FORTIS.51670127,bodyshop external site,amoe,Assistance à la maîtrise d'oeuvre,CONSU,Consultants,...,Belgium,EMEA,FORTIS.51670127FORTIS.147081,2017,<= 100K€,-,-,-,98.34532,bnpp fortis ; bodyshop external site ; freebor...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133303,IT033,banca nazionale del lavoro,IT,BNL.2010000009.IT033,BNL.2010000009,debiti vs fornitori per fatture da ricev.,???,Not affected to a precise family,???,ZZ-Not affected to a precise family,...,Italy,EMEA,BNL.2010000009BNL.3724,2016,<= 100K€,-,-,-,0.00000,banca nazionale del lavoro ; debiti vs fornito...
133304,IT033,banca nazionale del lavoro,IT,BNL.2010000008.IT033,BNL.2010000008,c/anticipi immobili affitti e condomini,???,Not affected to a precise family,???,ZZ-Not affected to a precise family,...,Italy,EMEA,BNL.2010000008BNL.62506,2016,<= 100K€,-,-,-,0.00000,banca nazionale del lavoro ; c/anticipi immobi...
133305,IT033,banca nazionale del lavoro,IT,BNL.1030000001.IT033,BNL.1030000001,transitorio pagamenti,???,Not affected to a precise family,???,ZZ-Not affected to a precise family,...,Italy,EMEA,BNL.1030000001BNL.1001669,2016,<= 100K€,-,-,-,0.00000,banca nazionale del lavoro ; transitorio pagam...
133306,IT033,banca nazionale del lavoro,IT,BNL.2010000008.IT033,BNL.2010000008,c/anticipi immobili affitti e condomini,???,Not affected to a precise family,???,ZZ-Not affected to a precise family,...,Italy,EMEA,BNL.2010000008BNL.53939,2016,<= 100K€,-,-,-,0.00000,banca nazionale del lavoro ; c/anticipi immobi...


In [162]:
df1617["LR_predict"], df1617["PAC_predict"], df1617["SGD_predict"], df1617["SVC_predict"] = preditStream(vecteur1821, df1617["AcléEt"], modelLR1821, modelPAC1821, modelSGDC1821, modelLSVC1821)

2021-05-19 23:03:19.035384 - temps de traitement du transform : 12.30899715423584
2021-05-19 23:03:20.678514 - temps de prediction de modelLRRun : 1.6420342922210693
2021-05-19 23:03:22.184540 - temps de prediction de modelPACRun : 1.504920482635498
2021-05-19 23:03:23.664582 - temps de prediction de modelSGDCRun : 1.4770991802215576
2021-05-19 23:03:24.882756 - temps de prediction de modelLSVCRun : 1.2181739807128906


In [163]:
df1617.to_excel('2016-2017 lignes streamées.xlsx')

In [ ]:
modelLRRun = LogisticRegression(solver='saga')
#l'apprentissage à proprement parler
modelLRRun.fit(X_RUN, dfW["Family Stream"])
#modelLRRun.fit(X_RUN, dfW["Acible"])

In [ ]:
labelsRun = modelLRRun.classes_

In [ ]:
#applique l'apprentissage aux autres de 2021Q1
dfORun = dfR

indexNames = dfORun[(dfORun['Family Stream'] != 'Others') | ((dfORun['Family Stream'] == 'Others') & (dfORun['Category Stream'] == 'Transaction Fees'))].index
dfORun.drop(indexNames , inplace=True)

dfORun.fillna("-", inplace=True)

In [ ]:
dfORun["AcléEt"]=dfORun["Entité"]+" ; "+dfORun["Compte"]+" ; "+dfORun["Fournisseur identification"]+" ; "+dfORun["Cd_Scat"]+" ; "+dfORun["Fournisseur"]

In [ ]:
to_predictRun = vectRun.transform(dfORun["AcléEt"])

In [ ]:
dfORun["predictOthers 1"] = modelLRRun.predict(to_predictRun)
predictProbaRun = modelLRRun.predict_proba(to_predictRun)
dfORun["predictOthersProba 1"] = [max(p) for p in predictProbaRun]

In [ ]:
dfORun["predictOthers 2"]=[{i:j for i,j in zip(x,labelsRun)}.get(sorted(x,reverse=True)[1]) for x in predictProbaRun]
dfORun["predictOthersProba 2"]=[sorted(x,reverse=True)[1] for x in predictProbaRun]
dfORun["predictOthers 3"]=[{i:j for i,j in zip(x,labelsRun)}.get(sorted(x,reverse=True)[2]) for x in predictProbaRun]
dfORun["predictOthersProba 3"]=[sorted(x,reverse=True)[2] for x in predictProbaRun]
dfORun["predictOthers 4"]=[{i:j for i,j in zip(x,labelsRun)}.get(sorted(x,reverse=True)[3]) for x in predictProbaRun]
dfORun["predictOthersProba 4"]=[sorted(x,reverse=True)[3] for x in predictProbaRun]
dfORun["predictOthers 5"]=[{i:j for i,j in zip(x,labelsRun)}.get(sorted(x,reverse=True)[4]) for x in predictProbaRun]
dfORun["predictOthersProba 5"]=[sorted(x,reverse=True)[4] for x in predictProbaRun]

In [ ]:
dfORun

In [ ]:
dfORun.to_excel('2021 Q1 Predicted Others with Proba with predict others only Family.xlsx')

In [59]:
df2018 = dfGT.copy()

df2018["Entité"] = df2018["Entité"].str.lower()
df2018["Compte"] = df2018["Compte"].str.lower()
df2018["Fournisseur identification"] = df2018["Fournisseur identification"].str.lower()
df2018["Cd_Scat"] = df2018["Cd_Scat"].str.lower()
df2018["Fournisseur"] = df2018["Fournisseur"].str.lower()

#remplis les vide avec un tiret "-"
df2018.fillna("-", inplace=True)

df2018["AcléEt"]=df2018["Entité"]+" ; "+df2018["Compte"]+" ; "+df2018["Fournisseur identification"]+" ; "+df2018["Cd_Scat"]+" ; "+df2018["Fournisseur"]
df2018

,CdEnt,Entité,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Cd_Cat,Catégorie_de_dépenses,Cd_Fam,Famille_de_dépenses,Fournisseur,Fournisseur identification,SommeDekEUR,Family Stream,Category Stream,Sub Category Stream,AcléEt
0,FR042,bnpp real estate,maintenance info,mtins,"Installation, maintenance",MTSRV,Services sur matériel informatiques,TECH,Technologie,octime,oct,6.35505,Technology,Hardware,-,bnpp real estate ; maintenance info ; oct ; mt...
1,RU41332,arval ooo,cost of services maintenance closed contract,main0,z-Coûts de maintenance - autres,MAINT,Coûts de maintenance,IMMO,Immobilier,"""??? """"???""""""","""??? """"???""""""",0.01574,Corporate Services,Corporate Real Estate,Maintenance,arval ooo ; cost of services maintenance close...
2,RU41332,arval ooo,employee car - fuel,carbu,Carburant,AUTO,Voitures,SPRO,Services professionnels,"""??? """"???""""""","""??? """"???""""""",0.26786,Corporate Services,General Services,Car Fleet,"arval ooo ; employee car - fuel ; ""??? """"???""""..."
3,RU41332,arval ooo,it cost - small equipments,mtins,"Installation, maintenance",MTSRV,Services sur matériel informatiques,TECH,Technologie,"""??? """"???""""""","""??? """"???""""""",0.01717,Technology,IT Services,-,"arval ooo ; it cost - small equipments ; ""??? ..."
4,RU41332,arval ooo,??????? ?? ???????????? ??,???,Non affecté à une famille précise,???,ZZ-Non affecté à une famille précise,???,ZZ-Non affecté,"""??? """"???""""""","""??? """"???""""""",6.24923,Others,-,-,"arval ooo ; ??????? ?? ???????????? ?? ; ""??? ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410131,TR43213,teb finansman as,travel expenses (domestic),voya0,z-Voyages - autres,VOYA0,Z-Voyages - autres,VOYAG,Voyages,-,-,0.00446,Others,-,-,teb finansman as ; travel expenses (domestic) ...
410132,TR43213,teb finansman as,travel expenses (abroad),hotel,Hôtels,HOTEL,Hôtels,VOYAG,Voyages,-,-,0.44563,Others,-,-,teb finansman as ; travel expenses (abroad) ; ...
410133,RU43677,cardif insurance company llc,opex marketing consulting and channels testing,c&mk0,z-Communication et Marketing - autres,C&MK0,Z-Communication et Marketing - autres,C&MK,Communication et Marketing,??? ???? ?????? ? ????????????????,??? ???? ?????? ? ????????????????,31.91862,Professional Services,Non IT Consultancy (excl. EMC),-,cardif insurance company llc ; opex marketing ...
410134,RU43677,cardif insurance company llc,opex other consulting,cons0,"z-Conseil, assistance technique - autres",CONSU,"Conseil, assistance technique",SPRO,Services professionnels,??? ???? ?????? ? ????????????????,??? ???? ?????? ? ????????????????,113.51913,Professional Services,Non IT Consultancy (excl. EMC),-,cardif insurance company llc ; opex other cons...


In [60]:
to_predictRun2018 = vectRun.transform(df2018["AcléEt"])

In [86]:
with Timer() as timer:
    df2018["LR_predict"] = modelLRRun.predict(to_predictRun2018)
print(str(datetime.now())+' - temps de prediction de modelLRRun : '+str(timer.interval))
with Timer() as timer:
    df2018["PAC_predict"] = modelPACRun.predict(to_predictRun2018)
print(str(datetime.now())+' - temps de prediction de modelPACRun : '+str(timer.interval))
with Timer() as timer:
    df2018["SGD_predict"] = modelSGDCRun.predict(to_predictRun2018)
print(str(datetime.now())+' - temps de prediction de modelSGDCRun : '+str(timer.interval))
with Timer() as timer:
    df2018["SVC_predict"] = modelLSVCRun.predict(to_predictRun2018)
print(str(datetime.now())+' - temps de prediction de modelLSVCRun : '+str(timer.interval))

2021-05-19 16:48:51.379866 - temps de prediction de modelLRRun : 3.4541256427764893
2021-05-19 16:48:55.165074 - temps de prediction de modelPACRun : 3.7842071056365967
2021-05-19 16:48:58.769172 - temps de prediction de modelSGDCRun : 3.602855920791626
2021-05-19 16:49:01.304569 - temps de prediction de modelLSVCRun : 2.534393787384033


In [62]:
df2018.to_excel('Lignes streamées 2018 sur apprentissage Full.xlsx')

In [83]:
df2018["Acible"]=df2018["Family Stream"]+" ; "+df2018["Category Stream"]+" ; "+df2018["Sub Category Stream"]

In [89]:
#print(classification_report(df2018["Acible"], df2018["LR_predict"]))

In [19]:
dfM = pd.read_excel('./Lignes à streamer.xlsx')

In [20]:
df2016 = dfM.copy()

df2016["Entité"] = df2016["Entité"].str.lower()
df2016["Compte"] = df2016["Compte"].str.lower()
df2016["Fournisseur identification"] = df2016["Fournisseur identification"].str.lower()
df2016["Cd_Scat"] = df2016["Cd_Scat"].str.lower()
df2016["Fournisseur"] = df2016["Fournisseur"].str.lower()

#remplis les vide avec un tiret "-"
df2016.fillna("-", inplace=True)

df2016["AcléEt"]=df2016["Entité"]+" ; "+df2016["Compte"]+" ; "+df2016["Fournisseur identification"]+" ; "+df2016["Cd_Scat"]+" ; "+df2016["Fournisseur"]
df2016

,CdEnt,Entité,CodePays,CdCompte,CLE_COMPTE,Compte,Cd_Scat,Expense_Sub_Category,Cd_Cat,Expense_Category,...,Key,Métier v2,Country,Region,Année,Amount Category,Family Stream,Category Stream,Sub Category Stream,AcléEt
0,FR8444,bddf - banque en ligne,FR,SAP_IMEX.607140.FR8444,SAP_IMEX.607140,sous traitance - services divers,spro0,z-Professional services - others,SPRO0,Z-Professional services - others,...,FR8444_SAP_IMEX.607140_SAP_IMEX.100059654,French Retail Banking,France,France,2016,<= 100K€,-,-,-,bddf - banque en ligne ; sous traitance - serv...
1,FR101,bddf - fonctions,FR,SAP_IMEX.607112.FR101,SAP_IMEX.607112,prestation d'assistance extérieure informatique,amoe,Assistance à la maîtrise d'oeuvre,CONSU,Consultants,...,FR101_SAP_IMEX.607112_SAP_IMEX.100038426,French Retail Banking,France,France,2016,<= 100K€,-,-,-,bddf - fonctions ; prestation d'assistance ext...
2,FR317,bddf - dr lyon,FR,SAP_IMEX.615440.FR317,SAP_IMEX.615440,réception - autres resto. - employés bnpp,resto,Restaurants,RESTO,Restaurants,...,FR317_SAP_IMEX.615440_SAP_IMEX.100025945,French Retail Banking,France,France,2016,<= 100K€,-,-,-,bddf - dr lyon ; réception - autres resto. - e...
3,FR513,bddf - entreprises,FR,SAP_IMEX.607140.FR513,SAP_IMEX.607140,sous traitance - services divers,spro0,z-Professional services - others,SPRO0,Z-Professional services - others,...,FR513_SAP_IMEX.607140_SAP_IMEX.100082376,French Retail Banking,France,France,2017,<= 100K€,-,-,-,bddf - entreprises ; sous traitance - services...
4,FR616,bddf - dsi,FR,SAP_IMEX.300105.FR616,SAP_IMEX.300105,frais à payer/mensualisation,cons0,z-Consultants - others,CONSU,Consultants,...,FR616_SAP_IMEX.300105_SAP_IMEX.100018665,French Retail Banking,France,France,2016,<= 100K€,-,-,-,bddf - dsi ; frais à payer/mensualisation ; in...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133309,LU45555,banque generale du luxembourg,LU,FORTIS.51680133.LU45555,FORTIS.51680133,dispatching supplies,post0,z-Postage - others,POSTE,Postage,...,LU45555_FORTIS.51680133_FORTIS.99059,BGL BNP Paribas,Luxembourg,EMEA,2017,<= 100K€,-,-,-,banque generale du luxembourg ; dispatching su...
133310,BE13394,bnpp fortis,BE,FORTIS.51675124.BE13394,FORTIS.51675124,general advice - expertise,strat,Strategic,CONSU,Consultants,...,BE13394_FORTIS.51675124_FORTIS.87157,BNP Paribas Fortis,Belgium,EMEA,2017,<= 100K€,-,-,-,bnpp fortis ; general advice - expertise ; ste...
133311,FR101,bddf - fonctions,FR,SAP_IMEX.634107.FR101,SAP_IMEX.634107,honoraires - prestations d'assistance technique,amoe,Assistance à la maîtrise d'oeuvre,CONSU,Consultants,...,FR101_SAP_IMEX.634107_SAP_IMEX.100026733,French Retail Banking,France,France,2016,<= 100K€,-,-,-,bddf - fonctions ; honoraires - prestations d'...
133312,FR513,bddf - entreprises,FR,SAP_IMEX.634107.FR513,SAP_IMEX.634107,honoraires - prestations d'assistance technique,amoe,Assistance à la maîtrise d'oeuvre,CONSU,Consultants,...,FR513_SAP_IMEX.634107_SAP_IMEX.100012280,French Retail Banking,France,France,2016,<= 100K€,-,-,-,bddf - entreprises ; honoraires - prestations ...


In [21]:
to_predictRun = vectRun.transform(df2016["AcléEt"])

In [22]:
with Timer() as timer:
    df2016["LR_predict"] = modelLRRun.predict(to_predictRun)
print(str(datetime.now())+' - temps de prediction de modelLRRun : '+str(timer.interval))
with Timer() as timer:
    df2016["PAC_predict"] = modelPACRun.predict(to_predictRun)
print(str(datetime.now())+' - temps de prediction de modelPACRun : '+str(timer.interval))
with Timer() as timer:
    df2016["SGD_predict"] = modelSGDCRun.predict(to_predictRun)
print(str(datetime.now())+' - temps de prediction de modelSGDCRun : '+str(timer.interval))
with Timer() as timer:
    df2016["SVC_predict"] = modelLSVCRun.predict(to_predictRun)
print(str(datetime.now())+' - temps de prediction de modelLSVCRun : '+str(timer.interval))

2021-05-18 00:38:51.576144 - temps de prediction de modelLRRun : 0.6569056510925293
2021-05-18 00:38:52.227721 - temps de prediction de modelPACRun : 0.6515772342681885
2021-05-18 00:38:52.878329 - temps de prediction de modelSGDCRun : 0.6506080627441406
2021-05-18 00:38:53.089695 - temps de prediction de modelLSVCRun : 0.2113659381866455


In [23]:
df2016.to_excel('Lignes streamées.xlsx')

In [66]:
to_predictRun20192020 = vectRun.transform(dfW["AcléEt"])

In [67]:
with Timer() as timer:
    dfW["LR_predict"] = modelLRRun.predict(to_predictRun20192020)
print(str(datetime.now())+' - temps de prediction de modelLRRun : '+str(timer.interval))
with Timer() as timer:
    dfW["PAC_predict"] = modelPACRun.predict(to_predictRun20192020)
print(str(datetime.now())+' - temps de prediction de modelPACRun : '+str(timer.interval))
with Timer() as timer:
    dfW["SGD_predict"] = modelSGDCRun.predict(to_predictRun20192020)
print(str(datetime.now())+' - temps de prediction de modelSGDCRun : '+str(timer.interval))
with Timer() as timer:
    dfW["SVC_predict"] = modelLSVCRun.predict(to_predictRun20192020)
print(str(datetime.now())+' - temps de prediction de modelLSVCRun : '+str(timer.interval))

2021-05-19 11:26:12.854753 - temps de prediction de modelLRRun : 7.421880722045898
2021-05-19 11:26:18.574665 - temps de prediction de modelPACRun : 5.719911575317383
2021-05-19 11:26:24.150217 - temps de prediction de modelSGDCRun : 5.575551986694336
2021-05-19 11:26:28.998011 - temps de prediction de modelLSVCRun : 4.847793817520142


In [68]:
dfW

,CdEnt,Entité,CodePays,CLE_COMPTE,Compte,Cd_Scat,Sous_catégorie_de_dépenses,Cd_Cat,Catégorie_de_dépenses,Cd_Fam,...,Category Stream,Sub Category Stream,AcléMin,AcléEt,AcléMax,Acible,LR_predict,PAC_predict,SGD_predict,SVC_predict
0,PL45551,bnpp bank polska,PL,ROD.954M30,organizacja eventów i spotka?,rp,"Evénementiel, RP et services marketing",CSLTG,Prestations de conseil en communication,C&MK,...,Hardware,-,bnpp bank polska ; organizacja eventów i spotk...,bnpp bank polska ; organizacja eventów i spotk...,bnpp bank polska;PL;organizacja eventów i spot...,Technology ; Hardware ; -,Technology ; Hardware ; -,Technology ; Hardware ; -,Technology ; Hardware ; -,Technology ; Hardware ; -
1,ES018,uci espana,ES,UCI.62110,alquiler locales,loyer,Loyer,LOYER,Coûts d'occupation,IMMO,...,Corporate Real Estate,Costs of occupancy,"uci espana ; alquiler locales ; ""3491 ma, s.l""","uci espana ; alquiler locales ; ""3491 ma, s.l""...",uci espana;ES;alquiler locales;Loyer;Coûts d'o...,Corporate Services ; Corporate Real Estate ; C...,Corporate Services ; Corporate Real Estate ; C...,Corporate Services ; Corporate Real Estate ; C...,Corporate Services ; Corporate Real Estate ; C...,Corporate Services ; Corporate Real Estate ; C...
2,FR084,bnpp personal finance,FR,OCAPP.634622,presta informatiques refacturees ips specifiqu...,amoe,Assistance à la maîtrise d'oeuvre,CONSU,"Conseil, assistance technique",SPRO,...,-,-,bnpp personal finance ; presta informatiques r...,bnpp personal finance ; presta informatiques r...,bnpp personal finance;FR;presta informatiques ...,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -
3,FR060,bnpp imex,FR,SAP_IMEX.I621100RE0,loyers locaux d'exp,loyer,Loyer,LOYER,Coûts d'occupation,IMMO,...,-,-,bnpp imex ; loyers locaux d'exp ; bnp paribas ...,bnpp imex ; loyers locaux d'exp ; bnp paribas ...,bnpp imex;FR;loyers locaux d'exp;Loyer;Coûts d...,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -
4,FR084,bnpp personal finance,FR,OCAPP.634622,presta informatiques refacturees ips specifiqu...,amoe,Assistance à la maîtrise d'oeuvre,CONSU,"Conseil, assistance technique",SPRO,...,-,-,bnpp personal finance ; presta informatiques r...,bnpp personal finance ; presta informatiques r...,bnpp personal finance;FR;presta informatiques ...,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -,BNP PARIBAS GROUP ; - ; -
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1059375,ZA62590,rcs home loans proprietary ltd,-,-,postage and courier,post0,z-Affranchissement et courrier - autres,POSTE,Affranchissement et courrier,C&MG,...,General Services,Postage,rcs home loans proprietary ltd ; postage and c...,rcs home loans proprietary ltd ; postage and c...,rcs home loans proprietary ltd;-;postage and c...,Corporate Services ; General Services ; Postage,Corporate Services ; General Services ; Postage,Corporate Services ; General Services ; Postage,Corporate Services ; General Services ; Postage,Corporate Services ; General Services ; Postage
1059376,ZA62590,rcs home loans proprietary ltd,-,-,ict - networks,cons0,"z-Conseil, assistance technique - autres",CONSU,"Conseil, assistance technique",SPRO,...,IT Services,-,rcs home loans proprietary ltd ; ict - network...,rcs home loans proprietary ltd ; ict - network...,rcs home loans proprietary ltd;-;ict - network...,Technology ; IT Services ; -,Technology ; IT Services ; -,Technology ; IT Services ; -,Technology ; IT Services ; -,Technology ; IT Services ; -
1059377,ZA62590,rcs home loans proprietary ltd,-,-,ict - networks,cons0,"z-Conseil, assistance technique - autres",CONSU,"Conseil, assistance technique",SPRO,...,IT Services,-,rcs home loans proprietary ltd ; ict - network...,rcs home loans proprietary ltd ; ict - network...,rcs home loans proprietary ltd;-;ic

In [70]:
len(dfW)

1059380

In [72]:
dfW[0:int(len(dfW)/2)].to_excel('Lignes streamées 2019 2020 sur apprentissage Full - moitié 1.xlsx')

In [73]:
dfW[int(len(dfW)/2):].to_excel('Lignes streamées 2019 2020 sur apprentissage Full - moitié 2.xlsx')

# TREE/FOREST

In [ ]:
dicModelDT={}

In [ ]:
modelDT = DT(max_depth = 50)
dicModelDT[50]=modelDT.fit(X_train_vectorized, y_train)

#instanciation
#model_Tree = DT()
#training
#model_Tree.fit_transform(x_train, y_train)
#calcule de précision
#print( model_Tree.score( x_test, y_test))

In [ ]:
predictionsDT = modelDT.predict(X_test_vectorized)

In [ ]:
toCompareDT=pd.DataFrame([predictionsDT,y_test])
toCompareDT = toCompareDT.transpose()
toCompareDT["score"]=[1 if i == j else 0 for i,j in zip (toCompareDT[0], toCompareDT[1])]

100*toCompareDT["score"].sum()/toCompareDT["score"].shape

In [ ]:
predictionsDT

# KNN

In [ ]:
dicModelKNN={}

In [ ]:
modelKNN = KNNI(n_neighbors = 10)
dicModelKNN[10]=modelKNN.fit(X_train_vectorized.toarray(), y_train)

#instanciation
#knn = KNNI(n_neighbors = 3)
#training
#knn.fit_transform(x_train,y_train)
#calcule de précision
#print( knn.score( x_test, y_test))

In [ ]:
predictionsKNN = modelKNN.predict(X_train_vectorized)

In [ ]:
toCompareKNN=pd.DataFrame([predictionsKNN,y_test])
toCompareKNN = toCompareKNN.transpose()
toCompareKNN["score"]=[1 if i == j else 0 for i,j in zip (toCompareKNN[0], toCompareKNN[1])]

100*toCompareKNN["score"].sum()/toCompareKNN["score"].shape

# SVM

## LinearSVC

In [ ]:
dicModelSVM={}

In [ ]:
#modelSVM = make_pipeline(StandardScaler(with_mean=False), LinearSVC(max_iter = 1000,C=5.5))
for this_C in [1, 3, 6, 10, 30, 60, 100]:
    label = this_penalty+"_"+str(this_C)
    print("C : "+str(this_C)+" --> "+label)
    modelSVM = LinearSVC(max_iter = 200, C=this_C)#(max_iter = 1000, C=5.5)
    dicModelSVM[label]=modelSVM.fit(X_train_vectorized, y_train)

    print("    score train : "+str(dicModelSVM[label].score(X_train_vectorized,y_train)))
    print("    score test : "+str(dicModelSVM[label].score(X_test_vectorized,y_test)))

In [ ]:
for this_C in [1, 3, 6, 10, 30, 60, 100]:
    label = this_penalty+"_"+str(this_C)
    print("C : "+str(this_C)+" --> "+label)

    print("    score train : "+str(dicModelSVM[label].score(X_train_vectorized,y_train)))
    print("    score test : "+str(dicModelSVM[label].score(X_test_vectorized,y_test)))

In [ ]:
dump(dicModelSVM, 'modelesSVMLinear.modele')

In [ ]:
#print(dicModelSVM[10000].score(X_train_vectorized, y_train))
#print(dicModelSVM[5000].score(X_train_vectorized, y_train))
#print("1000 : "+str(dicModelSVM[1000].score(X_train_vectorized, y_train)))
print("1003    : "+str(dicModelSVM[1003].score(X_train_vectorized, y_train)))
print("1005.5  : "+str(dicModelSVM[1005.5].score(X_train_vectorized, y_train)))
print("1005.52 : "+str(dicModelSVM[1005.5].score(X_train_vectorized, y_train)))
print("1010    : "+str(dicModelSVM[1010].score(X_train_vectorized, y_train)))
#print("1030 : "+str(dicModelSVM[1030].score(X_train_vectorized, y_train)))
print()
#print(dicModelSVM[10000].score(X_test_vectorized,y_test))
#print(dicModelSVM[5000].score(X_test_vectorized,y_test))
#print("1000 : "+str(dicModelSVM[1000].score(X_test_vectorized,y_test)))
print("1003    : "+str(dicModelSVM[1003].score(X_test_vectorized,y_test)))
print("1005.5  : "+str(dicModelSVM[1005.5].score(X_test_vectorized,y_test)))
print("1005.52 : "+str(dicModelSVM[1005.5].score(X_test_vectorized,y_test)))
print("1010    : "+str(dicModelSVM[1010].score(X_test_vectorized,y_test)))
#print("1030 : "+str(dicModelSVM[1030].score(X_test_vectorized,y_test)))

In [ ]:
predictionsSVM = modelSVM.predict(X_test_vectorized)
predictionsProbaSVM = modelSVM.decision_function(X_test_vectorized)

In [ ]:
predictionsSVM

In [ ]:
predictionsProbaSVM[1]

In [ ]:
modelSVM.classes_

## SVC

In [ ]:
dicModelSVC={}

In [ ]:
modelSVC = SVC(max_iter = 1000, C=5.5)#, class_weight='balanced')

dicModelSVC[1005.5]=modelSVC.fit(X_train_vectorized, y_train)

In [ ]:
print("1005.5  : "+str(dicModelSVC[1005.5].score(X_train_vectorized, y_train)))
print()
print("1005.5  : "+str(dicModelSVC[1005.5].score(X_test_vectorized,y_test)))

In [ ]:
from adspy_shared_utilities import plot_class_regions_for_classifier_subplot
import matplotlib.pyplot as plt

fig, subaxes = plt.subplots(3, 4, figsize=(15, 10), dpi=50)

title = 'gamma = {:.2f}, C = {:.2f}'.format('scale', 5.5)
#clf = SVC(kernel = 'rbf', gamma = this_gamma, C = this_C).fit(X_train, y_train)
plot_class_regions_for_classifier_subplot(clf, X_train, y_train,
                                        X_test, y_test, title,
                                        subplot)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

# Boosting

In [ ]:
dicModelBoost={}
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
modelBoost = AdaBoostClassifier(n_estimators = 50)

dicModelBoost[50]=modelBoost.fit(X_train_vectorized, y_train)

In [ ]:
print("50  : "+str(dicModelBoost[50].score(X_train_vectorized, y_train)))
print()
print("50  : "+str(dicModelBoost[50].score(X_test_vectorized,y_test)))

# Bagging

In [ ]:
dicModelBagg={}
from sklearn.ensemble import BaggingClassifier

In [ ]:
modelBagg = BaggingClassifier(n_estimators = 50, oob_score= False)

dicModelBagg[1010]=modelBagg.fit(X_train_vectorized, y_train)

In [ ]:
print("1010  : "+str(dicModelBagg[1010].score(X_train_vectorized, y_train)))
print()
print("1010  : "+str(dicModelBagg[1010].score(X_test_vectorized,y_test)))

# Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier
dicModelNN={}

In [ ]:
units = 10
this_activation = 'relu'
this_alpha = 0.0001
modelNN = MLPClassifier(solver='adam', activation = this_activation, alpha = this_alpha, hidden_layer_sizes = [units, units, units], learning_rate='adaptive')
dicModelNN[1]=modelNN.fit(X_train_vectorized, y_train)

print("    score train : "+str(dicModelNN[1].score(X_train_vectorized,y_train)))
print("    score test : "+str(dicModelNN[1].score(X_test_vectorized,y_test)))

dump(dicModelNN, 'modelesNN_3_10.modele')

In [ ]:
dump(dicModelNN, 'modelesNN_3_10.modele')

In [ ]:
units = 2
this_activation = 'logistic'
this_alpha = 0.0001
modelNN = MLPClassifier(solver='adam', activation = this_activation, alpha = this_alpha, hidden_layer_sizes = [units], learning_rate='adaptive')
dicModelNN[2]=modelNN.fit(X_train_vectorized, y_train)

print("    score train : "+str(dicModelNN[2].score(X_train_vectorized,y_train)))
print("    score test : "+str(dicModelNN[2].score(X_test_vectorized,y_test)))

In [ ]:
for units in [1, 10, 100]:
    for this_activation in ['logistic', 'tanh', 'relu']:
        for this_alpha in [0.0001, 0.001, 0.01, 0.1, 1.0]:
            label = units+"_"+str(this_activation)+"_"+str(this_alpha)
            print("units : "+str(units)+" _ activation : "+this_activation+" _ alpha : "+str(this_alpha)+" --> "+label)
            modelNN = MLPClassifier(solver='adam', activation = this_activation, alpha = this_alpha, hidden_layer_sizes = [units, units, units], learning_rate='adaptive')
            dicModelNN[label]=modelNN.fit(X_train_vectorized, y_train)

            print("    score train : "+str(dicModelNN[label].score(X_train_vectorized,y_train)))
            print("    score test : "+str(dicModelNN[label].score(X_test_vectorized,y_test)))